In [14]:
import functools
import numpy as np
import pandas as pd
from scipy.stats import kstest
import matplotlib.pyplot as plt
import pylab as pl
import psycopg2
%matplotlib inline
import datetime
plt.style.use('ggplot')
from concurrent.futures import ProcessPoolExecutor

In [15]:
# create a database connection
sqluser = ''
dbname = ''
schema_name = ''

# define AKI

In [ ]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password='eicu')
cur = con.cursor()
cur.execute('SET search_path to ' + 'BP_project')

query = \
"""
DROP MATERIALIZED VIEW IF EXISTS AKI_creat;
CREATE MATERIALIZED VIEW AKI_creat AS

WITH tempo AS
  ( SELECT patientunitstayid,
           labname,
           labresultoffset,
           labresult,
           ROW_NUMBER() OVER (PARTITION BY patientunitstayid, labname ORDER BY labresultoffset ASC) AS POSITION
   FROM eicu.lab
   WHERE ((labname) = 'creatinine')
     AND labresultoffset BETWEEN -720 AND 720 -- first creat available value between -12 and +12h from admission 
     ORDER BY patientunitstayid, labresultoffset ),
     
base_creat as (
SELECT patientunitstayid,
       max(CASE WHEN (labname) = 'creatinine' AND POSITION =1 THEN labresult ELSE NULL END) AS creat1,
       max(CASE WHEN (labname) = 'creatinine' AND POSITION =1 THEN labresultoffset ELSE NULL END) AS creat1offset
FROM tempo
GROUP BY patientunitstayid
ORDER BY patientunitstayid
),

peakcr AS (SELECT * 
         FROM   (SELECT patientunitstayid, 
                        labresultoffset AS peakcreat7d_offset, 
                        labresult AS peakcreat7d, 
                        Row_number() 
                          OVER ( 
                            partition BY patientunitstayid 
                            ORDER BY lab.labresult DESC) AS position 
                 FROM   eicu.lab 
                 WHERE  labname LIKE 'creatinine%' 
                        AND labresultoffset >= 0 
                        AND labresultoffset <= 10080 
                 GROUP  BY patientunitstayid, 
                           labresultoffset, 
                           labresult 
                 ORDER  BY patientunitstayid, 
                           labresultoffset) AS temp 
         WHERE  position = 1),
         
peak_7d as (SELECT pt.patientunitstayid, 
       peakcreat7d, 
       peakcreat7d_offset, 
       ( pt.unitdischargeoffset - peakcreat7d_offset ) AS 
       peakcreat7d_to_discharge_offsetgap 
FROM   eicu.patient pt 
       LEFT OUTER JOIN peakcr 
                    ON peakcr.patientunitstayid = pt.patientunitstayid 
ORDER  BY pt.patientunitstayid ),

peakcr_48 AS
  (SELECT patientunitstayid,
          labresultoffset AS peakcreat48h_offset,
          labresult AS peakcreat48h,
          Row_number() OVER (PARTITION BY patientunitstayid
                             ORDER BY lab.labresult DESC) AS POSITION
   FROM eicu.lab
   WHERE labname LIKE 'creatinine%'
     AND labresultoffset >= 0
     AND labresultoffset <= (48 * 60) --Within 48hrs

   GROUP BY patientunitstayid,
            labresultoffset,
            labresult
   ORDER BY patientunitstayid,
            labresultoffset),
            
peak_48h as (
SELECT patientunitstayid
, peakcreat48h_offset
, peakcreat48h
FROM peakcr_48
WHERE POSITION = 1
),

exclude_chronic as (
SELECT
  DISTINCT treatment.patientunitstayid
FROM
  eicu.treatment
WHERE
  LOWER(treatment.treatmentstring) LIKE ANY ('{%rrt%,%dialysis%,%ultrafiltration%,%cavhd%,%cvvh%,%sled%}')
  AND 
  LOWER(treatment.treatmentstring) LIKE '%chronic%'
),

creat_final as (
select base_creat.patientunitstayid, base_creat.creat1, peak_7d.peakcreat7d, peak_48h.peakcreat48h

from base_creat left join peak_7d
on base_creat.patientunitstayid =peak_7d.patientunitstayid

left join peak_48h
on base_creat.patientunitstayid =peak_48h.patientunitstayid

where base_creat.patientunitstayid is not null and (peak_7d.patientunitstayid is not null or 
peak_48h.patientunitstayid is not null) 
),

raw as (
select creat_final.patientunitstayid, creat_final.creat1, creat_final.peakcreat7d,creat_final.peakcreat48h
, case when peakcreat7d is not null and (peakcreat7d/creat1)>1.5 then 1 else 0 end as c_7d
, case when peakcreat48h is not null and (peakcreat48h-creat1)>0.3 then 1 else 0 end as c_48h
from creat_final
),

aki_raw as (
select raw.patientunitstayid, raw.c_7d, raw.c_48h,
case when (c_7d +c_48h)>=1 then 1 else 0 end as AKI
from raw
order by patientunitstayid
)

select aki_raw.patientunitstayid, aki_raw.aki
from aki_raw
where aki_raw.patientunitstayid NOT IN (select exclude_chronic.patientunitstayid from exclude_chronic);
"""

data = pd.read_sql_query(query,con)
data.head()

# vitals, BP, HR, respiratoryrate and spo2

In [10]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password='eicus')
cur = con.cursor()
cur.execute('SET search_path to ' + 'public')

query1 = \
"""
select * from pivoted_vital
limit 10
"""

vassp = pd.read_sql_query(query1,con)
vassp.head()


,patientunitstayid,chartoffset,entryoffset,heartrate,respiratoryrate,spo2,nibp_systolic,nibp_diastolic,nibp_mean,temperature,temperaturelocation,ibp_systolic,ibp_diastolic,ibp_mean
0,141168,6,6,140.0,None,NaN,82.0,59.0,None,None,None,None,None,67.0
1,141168,21,21,70.0,None,94.0,NaN,NaN,None,None,None,None,None,75.0
2,141168,36,36,70.0,None,94.0,105.0,59.0,None,None,None,None,None,75.0
3,141168,51,51,140.0,None,94.0,103.0,65.0,None,None,None,None,None,80.0
4,141168,66,66,140.0,None,92.0,99.0,68.0,None,None,None,None,None,80.0


# sepsis

In [ ]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password='eicus')
cur = con.cursor()
cur.execute('SET search_path to ' + 'public')

query1 = \
"""
with sepsis_raw as (
select
  patientunitstayid
, case
  when apacheadmissiondx in ('Angina, unstable (angina interferes w/quality of life or meds are tolerated poorly)', 'Infarction, acute myocardial (MI)', 'MI admitted > 24 hrs after onset of ischemia')
    then 'ACS'
  when apacheadmissiondx in ('Chest pain, atypical (noncardiac chest pain)', 'Chest pain, epigastric', 'Chest pain, musculoskeletal', 'Chest pain, respiratory', 'Chest pain, unknown origin')
    then 'ChestPainUnknown'
  when apacheadmissiondx in ('Cardiomyopathy', 'CHF, congestive heart failure', 'Shock, cardiogenic')
    then 'CHF'
  when apacheadmissiondx in ('Angina, stable (asymp or stable pattern of symptoms w/meds)', 'Anomaly, cardiac congenital', 'Arteriovenous malformation, surgery for', 'Atrial Septal Defect (ASD) Repair', 'Cardiovascular medical, other', 'Cardiovascular surgery, other', 'Congenital Defect Repair (Other)')
    then 'CVOther'
  when apacheadmissiondx in ('Contusion, myocardial (include r/o)', 'Efffusion, pericardial', 'Endocarditis', 'Hypertension-pulmonary, primary/idiopathic', 'Monitoring, hemodynamic (pre-operative evaluation)', 'Pericardial effusion/tamponade', 'Pericardiectomy (total/subtotal)', 'Pericarditis', 'Tamponade, pericardial', 'Thrombus, arterial', 'Vascular medical, other', 'Vascular surgery, other')
    then 'CVOther'
  when apacheadmissiondx in ('Cardiac arrest (with or without respiratory arrest; for respiratory arrest see Respiratory System)', 'Rhythm disturbance (atrial, supraventricular)', 'Rhythm disturbance (conduction defect)', 'Rhythm disturbance (ventricular)')
    then 'CardiacArrest'
  when apacheadmissiondx in ('Ablation or mapping of cardiac conduction pathway', 'Defibrillator, automatic implantable cardiac; insertion of')
    then 'CVOther'
  when apacheadmissiondx in ('CABG alone, coronary artery bypass grafting', 'CABG alone, redo', 'CABG redo with other operation', 'CABG redo with valve repair/replacement', 'CABG with aortic valve replacement', 'CABG with double valve repair/replacement', 'CABG with mitral valve repair', 'CABG with mitral valve replacement', 'CABG with other operation', 'CABG with pulmonic or tricuspid valve repair or replacement ONLY.', 'CABG with single valve repair/replacement', 'CABG, minimally invasive; mid-CABG')
    then 'CABG'
  when apacheadmissiondx in ('Aortic and Mitral valve replacement', 'Aortic valve replacement (isolated)', 'Mitral valve repair', 'Mitral valve replacement', 'Papillary muscle rupture', 'Pulmonary valve surgery', 'Tricuspid valve surgery', 'Valve, double; repair/replacement', 'Valve, redo, single', 'Valve, single; repair/replacement', 'Valve, triple; repair/replacement')
    then 'ValveDz'
  when apacheadmissiondx in ('Pneumonia, aspiration', 'Pneumonia, bacterial', 'Pneumonia, fungal', 'Pneumonia, other', 'Pneumonia, parasitic (i.e., Pneumocystic pneumonia)', 'Pneumonia, viral')
    then 'PNA'
  when apacheadmissiondx in ('Apnea, sleep', 'Apnea-sleep; surgery for (i.e., UPPP - uvulopalatopharyngoplasty)', 'ARDS-adult respiratory distress syndrome, non-cardiogenic pulmonary edema', 'Arrest, respiratory (without cardiac arrest)')
    then 'RespMedOther'
  when apacheadmissiondx in ('Atelectasis', 'Biopsy, open lung', 'Effusions, pleural', 'Embolus, pulmonary', 'Guillain-Barre syndrome', 'Hemorrhage/hemoptysis, pulmonary', 'Hemothorax', 'Obstruction-airway (i.e., acute epiglottitis, post-extubation edema, foreign body, etc)', 'Pneumothorax', 'Respiratory - medical, other', 'Restrictive lung disease (i.e., Sarcoidosis, pulmonary fibrosis)', 'Tracheostomy', 'Weaning from mechanical ventilation (transfer from other unit or hospital only)')
    then 'RespMedOther'
  when apacheadmissiondx in ('Asthma', 'Emphysema/bronchitis')
    then 'Asthma-Emphys'
  when apacheadmissiondx in ('Bleeding, GI from esophageal varices/portal hypertension', 'Bleeding, GI-location unknown', 'Bleeding, lower GI', 'Bleeding, upper GI', 'Bleeding-lower GI, surgery for', 'Bleeding-other GI, surgery for', 'Bleeding-upper GI, surgery for', 'Bleeding-variceal, surgery for (excluding vascular shunting-see surgery for portosystemic shunt)', 'GI perforation/rupture', 'GI perforation/rupture, surgery for', 'Hemorrhage, intra/retroperitoneal', 'Ulcer disease, peptic')
    then 'GIBleed'
  when apacheadmissiondx in ('GI obstruction', 'GI obstruction, surgery for (including lysis of adhesions)')
    then 'GIObstruction'
  when apacheadmissiondx in ('CVA, cerebrovascular accident/stroke', 'Hemorrhage/hematoma, intracranial', 'Hemorrhage/hematoma-intracranial, surgery for', 'Hypertension, uncontrolled (for cerebrovascular accident-see Neurological System)', 'Subarachnoid hemorrhage/arteriovenous malformation', 'Subarachnoid hemorrhage/intracranial aneurysm', 'Subarachnoid hemorrhage/intracranial aneurysm, surgery for')
    then 'CVA'
  when apacheadmissiondx in ('Abscess, neurologic', 'Biopsy, brain', 'Hydrocephalus, obstructive', 'Neoplasm, neurologic', 'Neoplasm-cranial, surgery for (excluding transphenoidal)', 'Neoplasm-spinal cord, surgery or other related procedures', 'Neurologic medical, other', 'Neuromuscular medical, other', 'Palsy, cranial nerve', 'Seizures (primary-no structural brain disease)', 'Seizures-intractable, surgery for')
    then 'Neuro'
  when apacheadmissiondx in ('Coma/change in level of consciousness (for hepatic see GI, for diabetic see Endocrine, if related to cardiac arrest, see CV)', 'Nontraumatic coma due to anoxia/ischemia')
    then 'Coma'
  when apacheadmissiondx in ('Overdose, alcohols (bethanol, methanol, ethylene glycol)', 'Overdose, analgesic (aspirin, acetaminophen)', 'Overdose, antidepressants (cyclic, lithium)', 'Overdose, other toxin, poison or drug', 'Overdose, sedatives, hypnotics, antipsychotics, benzodiazepines', 'Overdose, self-inflicted', 'Overdose, street drugs (opiates, cocaine, amphetamine)', 'Toxicity, drug (i.e., beta blockers, calcium channel blockers, etc.)')
    then 'Overdose'
  when apacheadmissiondx in ('Sepsis, cutaneous/soft tissue', 'Sepsis, GI', 'Sepsis, gynecologic', 'Sepsis, other', 'Sepsis, pulmonary', 'Sepsis, renal/UTI (including bladder)', 'Sepsis, unknown')
    then 'Sepsis'
  when apacheadmissiondx in ('Renal failure, acute', 'Renal obstruction')
    then 'ARF'
  when apacheadmissiondx in ('Diabetic hyperglycemic hyperosmolar nonketotic coma (HHNC)', 'Diabetic ketoacidosis')
    then 'DKA'
  when apacheadmissiondx in ('Abdomen only trauma', 'Abdomen/extremity trauma', 'Abdomen/face trauma', 'Abdomen/multiple trauma', 'Abdomen/pelvis trauma', 'Abdomen/spinal trauma', 'Chest thorax only trauma', 'Chest/abdomen trauma', 'Chest/extremity trauma', 'Chest/face trauma', 'Chest/multiple trauma', 'Chest/pelvis trauma', 'Chest/spinal trauma', 'Chest/thorax only trauma', 'Extremity only trauma')
    then 'Trauma'
  when apacheadmissiondx in ('Extremity only trauma, surgery for', 'Extremity/face trauma', 'Extremity/face trauma, surgery for', 'Extremity/multiple trauma', 'Extremity/multiple trauma, surgery for', 'Face only trauma', 'Face only trauma, surgery for', 'Face/multiple trauma', 'Face/multiple trauma, surgery for', 'Facial surgery (if related to trauma, see Trauma)', 'Head only trauma', 'Head/abdomen trauma', 'Head/chest trauma')
    then 'Trauma'
  when apacheadmissiondx in ('Head/extremity trauma', 'Head/face trauma', 'Head/multiple trauma', 'Head/pelvis trauma', 'Head/spinal trauma', 'Pelvis/extremity trauma', 'Pelvis/face trauma', 'Pelvis/hip trauma', 'Pelvis/multiple trauma', 'Pelvis/spinal trauma', 'Spinal cord only trauma', 'Spinal/extremity trauma', 'Spinal/face trauma', 'Spinal/multiple trauma', 'Trauma medical, other', 'Trauma surgery, other')
    then 'Trauma'
else 'Other' end as apachedxgroup
, apacheadmissiondx
from eicu.patient
),

sepsis_data as (
select patientunitstayid, 
case when lower(apacheadmissiondx) like '%sepsis%' then 1 else 0 end as sepsis_binary
from sepsis_raw
group by patientunitstayid, apacheadmissiondx
)

select * from sepsis_data



"""

sepsis = pd.read_sql_query(query1,con)
sepsis.head()


# vasopressor

In [ ]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password='eicus')
cur = con.cursor()
cur.execute('SET search_path to ' + 'public')

query1 = \
"""

with tr as
(
  select
    patientunitstayid
   , treatmentoffset as chartoffset
   , max(case when treatmentstring in
   (
     'toxicology|drug overdose|vasopressors|vasopressin' --                                                                   |    23
   , 'toxicology|drug overdose|vasopressors|phenylephrine (Neosynephrine)' --                                                 |    21
   , 'toxicology|drug overdose|vasopressors|norepinephrine > 0.1 micrograms/kg/min' --                                        |    62
   , 'toxicology|drug overdose|vasopressors|norepinephrine <= 0.1 micrograms/kg/min' --                                       |    29
   , 'toxicology|drug overdose|vasopressors|epinephrine > 0.1 micrograms/kg/min' --                                           |     6
   , 'toxicology|drug overdose|vasopressors|epinephrine <= 0.1 micrograms/kg/min' --                                          |     2
   , 'toxicology|drug overdose|vasopressors|dopamine 5-15 micrograms/kg/min' --                                               |     7
   , 'toxicology|drug overdose|vasopressors|dopamine >15 micrograms/kg/min' --                                                |     3
   , 'toxicology|drug overdose|vasopressors' --                                                                               |    30
   , 'surgery|cardiac therapies|vasopressors|vasopressin' --                                                                  |   356
   , 'surgery|cardiac therapies|vasopressors|phenylephrine (Neosynephrine)' --                                                |  1000
   , 'surgery|cardiac therapies|vasopressors|norepinephrine > 0.1 micrograms/kg/min' --                                       |   390
   , 'surgery|cardiac therapies|vasopressors|norepinephrine <= 0.1 micrograms/kg/min' --                                      |   347
   , 'surgery|cardiac therapies|vasopressors|epinephrine > 0.1 micrograms/kg/min' --                                          |   117
   , 'surgery|cardiac therapies|vasopressors|epinephrine <= 0.1 micrograms/kg/min' --                                         |   178
   , 'surgery|cardiac therapies|vasopressors|dopamine  5-15 micrograms/kg/min' --                                             |   274
   , 'surgery|cardiac therapies|vasopressors|dopamine >15 micrograms/kg/min' --                                               |    23
   , 'surgery|cardiac therapies|vasopressors' --                                                                              |   596
   , 'renal|electrolyte correction|treatment of hypernatremia|vasopressin' --                                                 |     7
   , 'neurologic|therapy for controlling cerebral perfusion pressure|vasopressors|phenylephrine (Neosynephrine)' --           |   321
   , 'neurologic|therapy for controlling cerebral perfusion pressure|vasopressors|norepinephrine > 0.1 micrograms/kg/min' --  |   348
   , 'neurologic|therapy for controlling cerebral perfusion pressure|vasopressors|norepinephrine <= 0.1 micrograms/kg/min' -- |   374
   , 'neurologic|therapy for controlling cerebral perfusion pressure|vasopressors|epinephrine > 0.1 micrograms/kg/min' --     |    21
   , 'neurologic|therapy for controlling cerebral perfusion pressure|vasopressors|epinephrine <= 0.1 micrograms/kg/min' --    |   199
   , 'neurologic|therapy for controlling cerebral perfusion pressure|vasopressors|dopamine 5-15 micrograms/kg/min' --         |   277
   , 'neurologic|therapy for controlling cerebral perfusion pressure|vasopressors|dopamine > 15 micrograms/kg/min' --         |    20
   , 'neurologic|therapy for controlling cerebral perfusion pressure|vasopressors' --                                         |   172
   , 'gastrointestinal|medications|hormonal therapy (for varices)|vasopressin' --                                             |   964
   , 'cardiovascular|shock|vasopressors|vasopressin' --                                                                       | 11082
   , 'cardiovascular|shock|vasopressors|phenylephrine (Neosynephrine)' --                                                     | 13189
   , 'cardiovascular|shock|vasopressors|norepinephrine > 0.1 micrograms/kg/min' --                                            | 24174
   , 'cardiovascular|shock|vasopressors|norepinephrine <= 0.1 micrograms/kg/min' --                                           | 17467
   , 'cardiovascular|shock|vasopressors|epinephrine > 0.1 micrograms/kg/min' --                                               |  2410
   , 'cardiovascular|shock|vasopressors|epinephrine <= 0.1 micrograms/kg/min' --                                              |  2384
   , 'cardiovascular|shock|vasopressors|dopamine  5-15 micrograms/kg/min' --                                                  |  4822
   , 'cardiovascular|shock|vasopressors|dopamine >15 micrograms/kg/min' --                                                    |  1102
   , 'cardiovascular|shock|vasopressors' --                                                                                   |  9335
   , 'toxicology|drug overdose|agent specific therapy|beta blockers overdose|dopamine' --                             |    66
   , 'cardiovascular|ventricular dysfunction|inotropic agent|norepinephrine > 0.1 micrograms/kg/min' --                       |   537
   , 'cardiovascular|ventricular dysfunction|inotropic agent|norepinephrine <= 0.1 micrograms/kg/min' --                      |   411
   , 'cardiovascular|ventricular dysfunction|inotropic agent|epinephrine > 0.1 micrograms/kg/min' --                          |   274
   , 'cardiovascular|ventricular dysfunction|inotropic agent|epinephrine <= 0.1 micrograms/kg/min' --                         |   456
   , 'cardiovascular|shock|inotropic agent|norepinephrine > 0.1 micrograms/kg/min' --                                         |  1940
   , 'cardiovascular|shock|inotropic agent|norepinephrine <= 0.1 micrograms/kg/min' --                                        |  1262
   , 'cardiovascular|shock|inotropic agent|epinephrine > 0.1 micrograms/kg/min' --                                            |   477
   , 'cardiovascular|shock|inotropic agent|epinephrine <= 0.1 micrograms/kg/min' --                                           |   505
   , 'cardiovascular|shock|inotropic agent|dopamine <= 5 micrograms/kg/min' --                                        |  1103
   , 'cardiovascular|shock|inotropic agent|dopamine  5-15 micrograms/kg/min' --                                       |  1156
   , 'cardiovascular|shock|inotropic agent|dopamine >15 micrograms/kg/min' --                                         |   144
   , 'surgery|cardiac therapies|inotropic agent|dopamine <= 5 micrograms/kg/min' --                                   |   171
   , 'surgery|cardiac therapies|inotropic agent|dopamine  5-15 micrograms/kg/min' --                                  |    93
   , 'surgery|cardiac therapies|inotropic agent|dopamine >15 micrograms/kg/min' --                                    |     3
   , 'cardiovascular|myocardial ischemia / infarction|inotropic agent|norepinephrine > 0.1 micrograms/kg/min' --              |   688
   , 'cardiovascular|myocardial ischemia / infarction|inotropic agent|norepinephrine <= 0.1 micrograms/kg/min' --             |   670
   , 'cardiovascular|myocardial ischemia / infarction|inotropic agent|epinephrine > 0.1 micrograms/kg/min' --                 |   381
   , 'cardiovascular|myocardial ischemia / infarction|inotropic agent|epinephrine <= 0.1 micrograms/kg/min' --                |   357
   , 'cardiovascular|ventricular dysfunction|inotropic agent|dopamine <= 5 micrograms/kg/min' --                      |   886
   , 'cardiovascular|ventricular dysfunction|inotropic agent|dopamine  5-15 micrograms/kg/min' --                     |   649
   , 'cardiovascular|ventricular dysfunction|inotropic agent|dopamine >15 micrograms/kg/min' --                       |    86
   , 'cardiovascular|myocardial ischemia / infarction|inotropic agent|dopamine <= 5 micrograms/kg/min' --             |   346
   , 'cardiovascular|myocardial ischemia / infarction|inotropic agent|dopamine  5-15 micrograms/kg/min' --            |   520
   , 'cardiovascular|myocardial ischemia / infarction|inotropic agent|dopamine >15 micrograms/kg/min' --              |    54
  ) then 1 else 0 end)::SMALLINT as vasopressor
  from eicu.treatment
  group by patientunitstayid, treatmentoffset
),

vaso_raw as(
select
  patientunitstayid, chartoffset, vasopressor
from tr
where vasopressor = 1
order by patientunitstayid, chartoffset
)

select patientunitstayid, max(vasopressor) as vasopressor
from vaso_raw
group by patientunitstayid



"""

vassp = pd.read_sql_query(query1,con)
vassp.head()


# vasopresin, heparin, etc.

In [ ]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password='eicus')
cur = con.cursor()
cur.execute('SET search_path to ' + 'public')

query1 = \
"""
with vw0 as
(
  select
    patientunitstayid
    , infusionoffset
    -- TODO: need dopamine rate
    , max(case when drugname in
              (
                   'Dopamine'
                 , 'Dopamine ()'
                 , 'DOPamine MAX 800 mg Dextrose 5% 250 ml  Premix (mcg/kg/min)'
                 , 'Dopamine (mcg/hr)'
                 , 'Dopamine (mcg/kg/hr)'
                 , 'dopamine (mcg/kg/min)'
                 , 'Dopamine (mcg/kg/min)'
                 , 'Dopamine (mcg/min)'
                 , 'Dopamine (mg/hr)'
                 , 'Dopamine (ml/hr)'
                 , 'Dopamine (nanograms/kg/min)'
                 , 'DOPamine STD 15 mg Dextrose 5% 250 ml  Premix (mcg/kg/min)'
                 , 'DOPamine STD 400 mg Dextrose 5% 250 ml  Premix (mcg/kg/min)'
                 , 'DOPamine STD 400 mg Dextrose 5% 500 ml  Premix (mcg/kg/min)'
                 , 'Dopamine (Unknown)'
              )
              -- note: no rows found for inotropin
                then 1
              else null end
            ) as dopamine

    -- this like statement is pretty reliable - no false positives when I checked
    -- also catches the brand name dobutrex
    , max(case when lower(drugname) like '%dobu%' then 1 else null end) as dobutamine
    , max(case
              when drugname in
              (
                 'Norepinephrine'
               , 'Norepinephrine ()'
               , 'Norepinephrine MAX 32 mg Dextrose 5% 250 ml (mcg/min)'
               , 'Norepinephrine MAX 32 mg Dextrose 5% 500 ml (mcg/min)'
               , 'Norepinephrine (mcg/hr)'
               , 'Norepinephrine (mcg/kg/hr)'
               , 'Norepinephrine (mcg/kg/min)'
               , 'Norepinephrine (mcg/min)'
               , 'Norepinephrine (mg/hr)'
               , 'Norepinephrine (mg/kg/min)'
               , 'Norepinephrine (mg/min)'
               , 'Norepinephrine (ml/hr)'
               , 'Norepinephrine STD 32 mg Dextrose 5% 282 ml (mcg/min)'
               , 'Norepinephrine STD 32 mg Dextrose 5% 500 ml (mcg/min)'
               , 'Norepinephrine STD 4 mg Dextrose 5% 250 ml (mcg/min)'
               , 'Norepinephrine STD 4 mg Dextrose 5% 500 ml (mcg/min)'
               , 'Norepinephrine STD 8 mg Dextrose 5% 250 ml (mcg/min)'
               , 'Norepinephrine STD 8 mg Dextrose 5% 500 ml (mcg/min)'
               , 'Norepinephrine (units/min)'
               , 'Norepinephrine (Unknown)'
               , 'norepinephrine Volume (ml)'
               , 'norepinephrine Volume (ml) (ml/hr)'
               -- levophed
              , 'Levophed (mcg/kg/min)'
              , 'levophed  (mcg/min)'
              , 'levophed (mcg/min)'
              , 'Levophed (mcg/min)'
              , 'Levophed (mg/hr)'
              , 'levophed (ml/hr)'
              , 'Levophed (ml/hr)'
              , 'NSS with LEVO (ml/hr)'
              , 'NSS w/ levo/vaso (ml/hr)'
              )
          then 1 else 0 end) as norepinephrine
    , max(case
          when drugname in
          (
             'Phenylephrine'
           , 'Phenylephrine ()'
           , 'Phenylephrine  MAX 100 mg Sodium Chloride 0.9% 250 ml (mcg/min)'
           , 'Phenylephrine (mcg/hr)'
           , 'Phenylephrine (mcg/kg/min)'
           , 'Phenylephrine (mcg/kg/min) (mcg/kg/min)'
           , 'Phenylephrine (mcg/min)'
           , 'Phenylephrine (mcg/min) (mcg/min)'
           , 'Phenylephrine (mg/hr)'
           , 'Phenylephrine (mg/kg/min)'
           , 'Phenylephrine (ml/hr)'
           , 'Phenylephrine  STD 20 mg Sodium Chloride 0.9% 250 ml (mcg/min)'
           , 'Phenylephrine  STD 20 mg Sodium Chloride 0.9% 500 ml (mcg/min)'
           , 'Volume (ml) Phenylephrine'
           , 'Volume (ml) Phenylephrine ()'
           -- neosynephrine is a synonym
           , 'neo-synephrine (mcg/min)'
           , 'neosynephrine (mcg/min)'
           , 'Neosynephrine (mcg/min)'
           , 'Neo Synephrine (mcg/min)'
           , 'Neo-Synephrine (mcg/min)'
           , 'NeoSynephrine (mcg/min)'
           , 'NEO-SYNEPHRINE (mcg/min)'
           , 'Neosynephrine (ml/hr)'
           , 'neosynsprine'
           , 'neosynsprine (mcg/kg/hr)'
          )
        then 1 else 0 end) as phenylephrine
    , max(case
            when drugname in
            (
                 'EPI (mcg/min)'
               , 'Epinepherine (mcg/min)'
               , 'Epinephrine'
               , 'Epinephrine ()'
               , 'EPINEPHrine(Adrenalin)MAX 30 mg Sodium Chloride 0.9% 250 ml (mcg/min)'
               , 'EPINEPHrine(Adrenalin)STD 4 mg Sodium Chloride 0.9% 250 ml (mcg/min)'
               , 'EPINEPHrine(Adrenalin)STD 4 mg Sodium Chloride 0.9% 500 ml (mcg/min)'
               , 'EPINEPHrine(Adrenalin)STD 7 mg Sodium Chloride 0.9% 250 ml (mcg/min)'
               , 'Epinephrine (mcg/hr)'
               , 'Epinephrine (mcg/kg/min)'
               , 'Epinephrine (mcg/min)'
               , 'Epinephrine (mg/hr)'
               , 'Epinephrine (mg/kg/min)'
               , 'Epinephrine (ml/hr)'
            ) then 1 else 0 end)
          as epinephrine
    , max(case
            when drugname in
            (
                'Vasopressin'
              , 'Vasopressin ()'
              , 'Vasopressin 20 Units Sodium Chloride 0.9% 100 ml (units/hr)'
              , 'Vasopressin 20 Units Sodium Chloride 0.9% 250 ml (units/hr)'
              , 'Vasopressin 40 Units Sodium Chloride 0.9% 100 ml (units/hr)'
              , 'Vasopressin 40 Units Sodium Chloride 0.9% 100 ml (units/kg/hr)'
              , 'Vasopressin 40 Units Sodium Chloride 0.9% 100 ml (units/min)'
              , 'Vasopressin 40 Units Sodium Chloride 0.9% 100 ml (Unknown)'
              , 'Vasopressin 40 Units Sodium Chloride 0.9% 200 ml (units/min)'
              , 'Vasopressin (mcg/kg/min)'
              , 'Vasopressin (mcg/min)'
              , 'Vasopressin (mg/hr)'
              , 'Vasopressin (mg/min)'
              , 'vasopressin (ml/hr)'
              , 'Vasopressin (ml/hr)'
              , 'Vasopressin (units/hr)'
              , 'Vasopressin (units/kg/min)'
              , 'vasopressin (units/min)'
              , 'Vasopressin (units/min)'
              , 'VAsopressin (units/min)'
              , 'Vasopressin (Unknown)'
            ) then 1 else 0 end)
          as vasopressin
    , max(case when drugname in
      (
           'Milrinone'
         , 'Milrinone ()'
         , 'Milrinone (mcg/kg/hr)'
         , 'Milrinone (mcg/kg/min)'
         , 'Milrinone (ml/hr)'
         , 'Milrinone (Primacor) 40 mg Dextrose 5% 200 ml (mcg/kg/min)'
         , 'Milronone (mcg/kg/min)'
         , 'primacore (mcg/kg/min)'
      ) then 1 else 0 end)
      as milrinone
    , max(case when drugname in
      (
          'Hepain (ml/hr)'
        , 'Heparin'
        , 'Heparin ()'
        , 'Heparin 25,000 Unit/D5w 250 ml (ml/hr)'
        , 'Heparin 25000 Units Dextrose 5% 500 ml  Premix (units/hr)'
        , 'Heparin 25000 Units Dextrose 5% 500 ml  Premix (units/kg/hr)'
        , 'Heparin 25000 Units Dextrose 5% 950 ml  Premix (units/kg/hr)'
        , 'HEPARIN #2 (units/hr)'
        , 'Heparin 8000u/1L NS (ml/hr)'
        , 'Heparin-EKOS (units/hr)'
        , 'Heparin/Femoral Sheath   (units/hr)'
        , 'Heparin (mcg/kg/hr)'
        , 'Heparin (mcg/kg/min)'
        , 'Heparin (ml/hr)'
        , 'heparin (units/hr)'
        , 'Heparin (units/hr)'
        , 'HEPARIN (units/hr)'
        , 'Heparin (units/kg/hr)'
        , 'Heparin (Unknown)'
        , 'Heparin via sheath (units/hr)'
        , 'Left  Heparin (units/hr)'
        , 'NSS carrier heparin (ml/hr)'
        , 'S-Heparin (units/hr)'
        , 'Volume (ml) Heparin-heparin 25,000 units in 0.45 % sodium chloride 500 mL infusion'
        , 'Volume (ml) Heparin-heparin 25,000 units in 0.45 % sodium chloride 500 mL infusion (ml/hr)'
        , 'Volume (ml) Heparin-heparin 25,000 units in dextrose 500 mL infusion'
        , 'Volume (ml) Heparin-heparin 25,000 units in dextrose 500 mL infusion (ml/hr)'
        , 'Volume (ml) Heparin-heparin infusion 2 units/mL in 0.9% sodium chloride (ARTERIAL LINE)'
        , 'Volume (ml) Heparin-heparin infusion 2 units/mL in 0.9% sodium chloride (ARTERIAL LINE) (ml/hr)'
      ) then 1 else 0 end)
      as heparin
  from eicu.infusiondrug
  group by patientunitstayid, infusionoffset
),

vasopressor_raw as(
select
  patientunitstayid
  , infusionoffset as chartoffset
  , dopamine::SMALLINT as dopamine
  , dobutamine::SMALLINT as dobutamine
  , norepinephrine::SMALLINT as norepinephrine
  , phenylephrine::SMALLINT as phenylephrine
  , epinephrine::SMALLINT as epinephrine
  , vasopressin::SMALLINT as vasopressin
  , milrinone::SMALLINT as milrinone
  , heparin::SMALLINT as heparin
from vw0
-- at least one of our drugs should be non-zero
where dopamine = 1
OR dobutamine = 1
OR norepinephrine = 1
OR phenylephrine = 1
OR epinephrine = 1
OR vasopressin = 1
OR milrinone = 1
OR heparin = 1
order by patientunitstayid, infusionoffset
)

select * from vasopressor_raw
limit 10



"""

vassp = pd.read_sql_query(query1,con)
vassp.head()


# cohort demographic and everything except BP and HR

In [212]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password='eicus')
cur = con.cursor()
cur.execute('SET search_path to ' + 'public')

query1 = \
"""
with firstIcu as (
     SELECT  uniquepid
                , patienthealthsystemstayid as hadmid
                , patientunitstayid
                , row_number() OVER (PARTITION BY uniquepid, patienthealthsystemstayid ORDER BY unitvisitnumber) AS first_icu
     FROM public.icustay_detail
),

-- ############ first ICU admission ############ 
cohort_first as (
SELECT  uniquepid, hadmid, patientunitstayid
FROM  firstIcu
    WHERE first_icu = 1
),
-- ############ BP reading in the first 24h and has fulfilled criteria 1,2,3  ############
cohort as (
select cohort_first.patientunitstayid
from cohort_first inner join BP_project.BP_cohort
on cohort_first.patientunitstayid = BP_cohort.patientunitstayid
),


-- ############ demographics ############ 
demographics as (

select distinct (cohort.patientunitstayid), pa.age, pa.gender, pa.ethnicity, pa.unitType, pa.admissionHeight, pa.admissionweight, pa.dischargeweight
--, ap.apacheScore, ap.apacheVersion
, ap.actualICUMortality as ICU_mort, ap.actualICULOS as ICU_los
, ap.actualHospitalMortality as hosp_mort, ap.actualHospitalLOS as hosp_los
, ap.unabridgedActualVentdays as ventilation_duration_days


from cohort left join eicu.apachePatientResult ap
on cohort.patientunitstayid = ap.patientunitstayid

left join eicu.patient pa
on cohort.patientunitstayid = pa.patientunitstayid

order by cohort.patientunitstayid
),

-- ############ fluid intake output ############ 
fluid_raw as(
select patientUnitStayID, netTotal,intakeOutputOffset,
Row_number() OVER (PARTITION BY patientunitstayid
                             ORDER BY in_out.intakeOutputOffset DESC) AS POSITION
from public.intakeoutput in_out
),

fluid as (
select fluid_raw.patientunitstayid, fluid_raw.nettotal, intakeOutputOffset 
from fluid_raw
where fluid_raw.POSITION=1
order by patientunitstayid
),

-- ############ medical history ############ 
hist_raw as (
select patientUnitStayID, ICD9Code 
, case when ICD9Code like '%I10%' then 1 else 0 end as Hypertension
, case when ICD9Code like '%I48%' then 1 else 0 end as Atrial_Fibrillation
, case when ICD9Code like '%C80%' then 1 else 0 end as Cancer
, case when ICD9Code like '%I50%' then 1 else 0 end as CHF
, case when ICD9Code like '%N18%' then 1 else 0 end as CKD
, case when ICD9Code like '%K74%' then 1 else 0 end as CLD
, case when ICD9Code like '%J44%' then 1 else 0 end as COPD
, case when ICD9Code like '%E11%' then 1 else 0 end as Diabetes
, case when ICD9Code like '%I25%' then 1 else 0 end as IHD
, case when ICD9Code like '%I63%' then 1 else 0 end as Cebrovascular_Accident
from eicu.diagnosis

),

history as (
select patientUnitStayID
, max(Hypertension) as Hypertension
, max(Atrial_Fibrillation) as Atrial_Fibrillation
, max(Cancer) as Cancer
, max(CHF) as CHF
, max(CKD) as CKD
, max(CLD) as CLD
, max(COPD) as COPD
, max(Diabetes) as Diabetes
, max(IHD) as IHD
, max(Cebrovascular_Accident) as Cebrovascular_Accident
from hist_raw
group by patientUnitStayID
),
-- ############ sepsis ############ 
sepsis_raw as (
select
  patientunitstayid
, case
  when apacheadmissiondx in ('Angina, unstable (angina interferes w/quality of life or meds are tolerated poorly)', 'Infarction, acute myocardial (MI)', 'MI admitted > 24 hrs after onset of ischemia')
    then 'ACS'
  when apacheadmissiondx in ('Chest pain, atypical (noncardiac chest pain)', 'Chest pain, epigastric', 'Chest pain, musculoskeletal', 'Chest pain, respiratory', 'Chest pain, unknown origin')
    then 'ChestPainUnknown'
  when apacheadmissiondx in ('Cardiomyopathy', 'CHF, congestive heart failure', 'Shock, cardiogenic')
    then 'CHF'
  when apacheadmissiondx in ('Angina, stable (asymp or stable pattern of symptoms w/meds)', 'Anomaly, cardiac congenital', 'Arteriovenous malformation, surgery for', 'Atrial Septal Defect (ASD) Repair', 'Cardiovascular medical, other', 'Cardiovascular surgery, other', 'Congenital Defect Repair (Other)')
    then 'CVOther'
  when apacheadmissiondx in ('Contusion, myocardial (include r/o)', 'Efffusion, pericardial', 'Endocarditis', 'Hypertension-pulmonary, primary/idiopathic', 'Monitoring, hemodynamic (pre-operative evaluation)', 'Pericardial effusion/tamponade', 'Pericardiectomy (total/subtotal)', 'Pericarditis', 'Tamponade, pericardial', 'Thrombus, arterial', 'Vascular medical, other', 'Vascular surgery, other')
    then 'CVOther'
  when apacheadmissiondx in ('Cardiac arrest (with or without respiratory arrest; for respiratory arrest see Respiratory System)', 'Rhythm disturbance (atrial, supraventricular)', 'Rhythm disturbance (conduction defect)', 'Rhythm disturbance (ventricular)')
    then 'CardiacArrest'
  when apacheadmissiondx in ('Ablation or mapping of cardiac conduction pathway', 'Defibrillator, automatic implantable cardiac; insertion of')
    then 'CVOther'
  when apacheadmissiondx in ('CABG alone, coronary artery bypass grafting', 'CABG alone, redo', 'CABG redo with other operation', 'CABG redo with valve repair/replacement', 'CABG with aortic valve replacement', 'CABG with double valve repair/replacement', 'CABG with mitral valve repair', 'CABG with mitral valve replacement', 'CABG with other operation', 'CABG with pulmonic or tricuspid valve repair or replacement ONLY.', 'CABG with single valve repair/replacement', 'CABG, minimally invasive; mid-CABG')
    then 'CABG'
  when apacheadmissiondx in ('Aortic and Mitral valve replacement', 'Aortic valve replacement (isolated)', 'Mitral valve repair', 'Mitral valve replacement', 'Papillary muscle rupture', 'Pulmonary valve surgery', 'Tricuspid valve surgery', 'Valve, double; repair/replacement', 'Valve, redo, single', 'Valve, single; repair/replacement', 'Valve, triple; repair/replacement')
    then 'ValveDz'
  when apacheadmissiondx in ('Pneumonia, aspiration', 'Pneumonia, bacterial', 'Pneumonia, fungal', 'Pneumonia, other', 'Pneumonia, parasitic (i.e., Pneumocystic pneumonia)', 'Pneumonia, viral')
    then 'PNA'
  when apacheadmissiondx in ('Apnea, sleep', 'Apnea-sleep; surgery for (i.e., UPPP - uvulopalatopharyngoplasty)', 'ARDS-adult respiratory distress syndrome, non-cardiogenic pulmonary edema', 'Arrest, respiratory (without cardiac arrest)')
    then 'RespMedOther'
  when apacheadmissiondx in ('Atelectasis', 'Biopsy, open lung', 'Effusions, pleural', 'Embolus, pulmonary', 'Guillain-Barre syndrome', 'Hemorrhage/hemoptysis, pulmonary', 'Hemothorax', 'Obstruction-airway (i.e., acute epiglottitis, post-extubation edema, foreign body, etc)', 'Pneumothorax', 'Respiratory - medical, other', 'Restrictive lung disease (i.e., Sarcoidosis, pulmonary fibrosis)', 'Tracheostomy', 'Weaning from mechanical ventilation (transfer from other unit or hospital only)')
    then 'RespMedOther'
  when apacheadmissiondx in ('Asthma', 'Emphysema/bronchitis')
    then 'Asthma-Emphys'
  when apacheadmissiondx in ('Bleeding, GI from esophageal varices/portal hypertension', 'Bleeding, GI-location unknown', 'Bleeding, lower GI', 'Bleeding, upper GI', 'Bleeding-lower GI, surgery for', 'Bleeding-other GI, surgery for', 'Bleeding-upper GI, surgery for', 'Bleeding-variceal, surgery for (excluding vascular shunting-see surgery for portosystemic shunt)', 'GI perforation/rupture', 'GI perforation/rupture, surgery for', 'Hemorrhage, intra/retroperitoneal', 'Ulcer disease, peptic')
    then 'GIBleed'
  when apacheadmissiondx in ('GI obstruction', 'GI obstruction, surgery for (including lysis of adhesions)')
    then 'GIObstruction'
  when apacheadmissiondx in ('CVA, cerebrovascular accident/stroke', 'Hemorrhage/hematoma, intracranial', 'Hemorrhage/hematoma-intracranial, surgery for', 'Hypertension, uncontrolled (for cerebrovascular accident-see Neurological System)', 'Subarachnoid hemorrhage/arteriovenous malformation', 'Subarachnoid hemorrhage/intracranial aneurysm', 'Subarachnoid hemorrhage/intracranial aneurysm, surgery for')
    then 'CVA'
  when apacheadmissiondx in ('Abscess, neurologic', 'Biopsy, brain', 'Hydrocephalus, obstructive', 'Neoplasm, neurologic', 'Neoplasm-cranial, surgery for (excluding transphenoidal)', 'Neoplasm-spinal cord, surgery or other related procedures', 'Neurologic medical, other', 'Neuromuscular medical, other', 'Palsy, cranial nerve', 'Seizures (primary-no structural brain disease)', 'Seizures-intractable, surgery for')
    then 'Neuro'
  when apacheadmissiondx in ('Coma/change in level of consciousness (for hepatic see GI, for diabetic see Endocrine, if related to cardiac arrest, see CV)', 'Nontraumatic coma due to anoxia/ischemia')
    then 'Coma'
  when apacheadmissiondx in ('Overdose, alcohols (bethanol, methanol, ethylene glycol)', 'Overdose, analgesic (aspirin, acetaminophen)', 'Overdose, antidepressants (cyclic, lithium)', 'Overdose, other toxin, poison or drug', 'Overdose, sedatives, hypnotics, antipsychotics, benzodiazepines', 'Overdose, self-inflicted', 'Overdose, street drugs (opiates, cocaine, amphetamine)', 'Toxicity, drug (i.e., beta blockers, calcium channel blockers, etc.)')
    then 'Overdose'
  when apacheadmissiondx in ('Sepsis, cutaneous/soft tissue', 'Sepsis, GI', 'Sepsis, gynecologic', 'Sepsis, other', 'Sepsis, pulmonary', 'Sepsis, renal/UTI (including bladder)', 'Sepsis, unknown')
    then 'Sepsis'
  when apacheadmissiondx in ('Renal failure, acute', 'Renal obstruction')
    then 'ARF'
  when apacheadmissiondx in ('Diabetic hyperglycemic hyperosmolar nonketotic coma (HHNC)', 'Diabetic ketoacidosis')
    then 'DKA'
  when apacheadmissiondx in ('Abdomen only trauma', 'Abdomen/extremity trauma', 'Abdomen/face trauma', 'Abdomen/multiple trauma', 'Abdomen/pelvis trauma', 'Abdomen/spinal trauma', 'Chest thorax only trauma', 'Chest/abdomen trauma', 'Chest/extremity trauma', 'Chest/face trauma', 'Chest/multiple trauma', 'Chest/pelvis trauma', 'Chest/spinal trauma', 'Chest/thorax only trauma', 'Extremity only trauma')
    then 'Trauma'
  when apacheadmissiondx in ('Extremity only trauma, surgery for', 'Extremity/face trauma', 'Extremity/face trauma, surgery for', 'Extremity/multiple trauma', 'Extremity/multiple trauma, surgery for', 'Face only trauma', 'Face only trauma, surgery for', 'Face/multiple trauma', 'Face/multiple trauma, surgery for', 'Facial surgery (if related to trauma, see Trauma)', 'Head only trauma', 'Head/abdomen trauma', 'Head/chest trauma')
    then 'Trauma'
  when apacheadmissiondx in ('Head/extremity trauma', 'Head/face trauma', 'Head/multiple trauma', 'Head/pelvis trauma', 'Head/spinal trauma', 'Pelvis/extremity trauma', 'Pelvis/face trauma', 'Pelvis/hip trauma', 'Pelvis/multiple trauma', 'Pelvis/spinal trauma', 'Spinal cord only trauma', 'Spinal/extremity trauma', 'Spinal/face trauma', 'Spinal/multiple trauma', 'Trauma medical, other', 'Trauma surgery, other')
    then 'Trauma'
else 'Other' end as apachedxgroup
, apacheadmissiondx
from eicu.patient
),

sepsis_data as (
select patientunitstayid, 
case when lower(apacheadmissiondx) like '%sepsis%' then 1 else 0 end as sepsis_binary
from sepsis_raw
group by patientunitstayid, apacheadmissiondx
),

-- ############ vasopressor ############ 
tr as
(
  select
    patientunitstayid
   , treatmentoffset as chartoffset
   , max(case when treatmentstring in
   (
     'toxicology|drug overdose|vasopressors|vasopressin' --                                                                   |    23
   , 'toxicology|drug overdose|vasopressors|phenylephrine (Neosynephrine)' --                                                 |    21
   , 'toxicology|drug overdose|vasopressors|norepinephrine > 0.1 micrograms/kg/min' --                                        |    62
   , 'toxicology|drug overdose|vasopressors|norepinephrine <= 0.1 micrograms/kg/min' --                                       |    29
   , 'toxicology|drug overdose|vasopressors|epinephrine > 0.1 micrograms/kg/min' --                                           |     6
   , 'toxicology|drug overdose|vasopressors|epinephrine <= 0.1 micrograms/kg/min' --                                          |     2
   , 'toxicology|drug overdose|vasopressors|dopamine 5-15 micrograms/kg/min' --                                               |     7
   , 'toxicology|drug overdose|vasopressors|dopamine >15 micrograms/kg/min' --                                                |     3
   , 'toxicology|drug overdose|vasopressors' --                                                                               |    30
   , 'surgery|cardiac therapies|vasopressors|vasopressin' --                                                                  |   356
   , 'surgery|cardiac therapies|vasopressors|phenylephrine (Neosynephrine)' --                                                |  1000
   , 'surgery|cardiac therapies|vasopressors|norepinephrine > 0.1 micrograms/kg/min' --                                       |   390
   , 'surgery|cardiac therapies|vasopressors|norepinephrine <= 0.1 micrograms/kg/min' --                                      |   347
   , 'surgery|cardiac therapies|vasopressors|epinephrine > 0.1 micrograms/kg/min' --                                          |   117
   , 'surgery|cardiac therapies|vasopressors|epinephrine <= 0.1 micrograms/kg/min' --                                         |   178
   , 'surgery|cardiac therapies|vasopressors|dopamine  5-15 micrograms/kg/min' --                                             |   274
   , 'surgery|cardiac therapies|vasopressors|dopamine >15 micrograms/kg/min' --                                               |    23
   , 'surgery|cardiac therapies|vasopressors' --                                                                              |   596
   , 'renal|electrolyte correction|treatment of hypernatremia|vasopressin' --                                                 |     7
   , 'neurologic|therapy for controlling cerebral perfusion pressure|vasopressors|phenylephrine (Neosynephrine)' --           |   321
   , 'neurologic|therapy for controlling cerebral perfusion pressure|vasopressors|norepinephrine > 0.1 micrograms/kg/min' --  |   348
   , 'neurologic|therapy for controlling cerebral perfusion pressure|vasopressors|norepinephrine <= 0.1 micrograms/kg/min' -- |   374
   , 'neurologic|therapy for controlling cerebral perfusion pressure|vasopressors|epinephrine > 0.1 micrograms/kg/min' --     |    21
   , 'neurologic|therapy for controlling cerebral perfusion pressure|vasopressors|epinephrine <= 0.1 micrograms/kg/min' --    |   199
   , 'neurologic|therapy for controlling cerebral perfusion pressure|vasopressors|dopamine 5-15 micrograms/kg/min' --         |   277
   , 'neurologic|therapy for controlling cerebral perfusion pressure|vasopressors|dopamine > 15 micrograms/kg/min' --         |    20
   , 'neurologic|therapy for controlling cerebral perfusion pressure|vasopressors' --                                         |   172
   , 'gastrointestinal|medications|hormonal therapy (for varices)|vasopressin' --                                             |   964
   , 'cardiovascular|shock|vasopressors|vasopressin' --                                                                       | 11082
   , 'cardiovascular|shock|vasopressors|phenylephrine (Neosynephrine)' --                                                     | 13189
   , 'cardiovascular|shock|vasopressors|norepinephrine > 0.1 micrograms/kg/min' --                                            | 24174
   , 'cardiovascular|shock|vasopressors|norepinephrine <= 0.1 micrograms/kg/min' --                                           | 17467
   , 'cardiovascular|shock|vasopressors|epinephrine > 0.1 micrograms/kg/min' --                                               |  2410
   , 'cardiovascular|shock|vasopressors|epinephrine <= 0.1 micrograms/kg/min' --                                              |  2384
   , 'cardiovascular|shock|vasopressors|dopamine  5-15 micrograms/kg/min' --                                                  |  4822
   , 'cardiovascular|shock|vasopressors|dopamine >15 micrograms/kg/min' --                                                    |  1102
   , 'cardiovascular|shock|vasopressors' --                                                                                   |  9335
   , 'toxicology|drug overdose|agent specific therapy|beta blockers overdose|dopamine' --                             |    66
   , 'cardiovascular|ventricular dysfunction|inotropic agent|norepinephrine > 0.1 micrograms/kg/min' --                       |   537
   , 'cardiovascular|ventricular dysfunction|inotropic agent|norepinephrine <= 0.1 micrograms/kg/min' --                      |   411
   , 'cardiovascular|ventricular dysfunction|inotropic agent|epinephrine > 0.1 micrograms/kg/min' --                          |   274
   , 'cardiovascular|ventricular dysfunction|inotropic agent|epinephrine <= 0.1 micrograms/kg/min' --                         |   456
   , 'cardiovascular|shock|inotropic agent|norepinephrine > 0.1 micrograms/kg/min' --                                         |  1940
   , 'cardiovascular|shock|inotropic agent|norepinephrine <= 0.1 micrograms/kg/min' --                                        |  1262
   , 'cardiovascular|shock|inotropic agent|epinephrine > 0.1 micrograms/kg/min' --                                            |   477
   , 'cardiovascular|shock|inotropic agent|epinephrine <= 0.1 micrograms/kg/min' --                                           |   505
   , 'cardiovascular|shock|inotropic agent|dopamine <= 5 micrograms/kg/min' --                                        |  1103
   , 'cardiovascular|shock|inotropic agent|dopamine  5-15 micrograms/kg/min' --                                       |  1156
   , 'cardiovascular|shock|inotropic agent|dopamine >15 micrograms/kg/min' --                                         |   144
   , 'surgery|cardiac therapies|inotropic agent|dopamine <= 5 micrograms/kg/min' --                                   |   171
   , 'surgery|cardiac therapies|inotropic agent|dopamine  5-15 micrograms/kg/min' --                                  |    93
   , 'surgery|cardiac therapies|inotropic agent|dopamine >15 micrograms/kg/min' --                                    |     3
   , 'cardiovascular|myocardial ischemia / infarction|inotropic agent|norepinephrine > 0.1 micrograms/kg/min' --              |   688
   , 'cardiovascular|myocardial ischemia / infarction|inotropic agent|norepinephrine <= 0.1 micrograms/kg/min' --             |   670
   , 'cardiovascular|myocardial ischemia / infarction|inotropic agent|epinephrine > 0.1 micrograms/kg/min' --                 |   381
   , 'cardiovascular|myocardial ischemia / infarction|inotropic agent|epinephrine <= 0.1 micrograms/kg/min' --                |   357
   , 'cardiovascular|ventricular dysfunction|inotropic agent|dopamine <= 5 micrograms/kg/min' --                      |   886
   , 'cardiovascular|ventricular dysfunction|inotropic agent|dopamine  5-15 micrograms/kg/min' --                     |   649
   , 'cardiovascular|ventricular dysfunction|inotropic agent|dopamine >15 micrograms/kg/min' --                       |    86
   , 'cardiovascular|myocardial ischemia / infarction|inotropic agent|dopamine <= 5 micrograms/kg/min' --             |   346
   , 'cardiovascular|myocardial ischemia / infarction|inotropic agent|dopamine  5-15 micrograms/kg/min' --            |   520
   , 'cardiovascular|myocardial ischemia / infarction|inotropic agent|dopamine >15 micrograms/kg/min' --              |    54
  ) then 1 else 0 end)::SMALLINT as vasopressor
  from eicu.treatment
  group by patientunitstayid, treatmentoffset
),

vaso_raw as(
select
  patientunitstayid, chartoffset, vasopressor
from tr
where vasopressor = 1
order by patientunitstayid, chartoffset
),

vasopressor as (
select patientunitstayid, max(vasopressor) as vasopressor
from vaso_raw
group by patientunitstayid
),

-- ############ vasopressin and heparin ############ 
vw0 as
(
  select
    patientunitstayid
    , infusionoffset
    -- TODO: need dopamine rate
    , max(case when drugname in
              (
                   'Dopamine'
                 , 'Dopamine ()'
                 , 'DOPamine MAX 800 mg Dextrose 5% 250 ml  Premix (mcg/kg/min)'
                 , 'Dopamine (mcg/hr)'
                 , 'Dopamine (mcg/kg/hr)'
                 , 'dopamine (mcg/kg/min)'
                 , 'Dopamine (mcg/kg/min)'
                 , 'Dopamine (mcg/min)'
                 , 'Dopamine (mg/hr)'
                 , 'Dopamine (ml/hr)'
                 , 'Dopamine (nanograms/kg/min)'
                 , 'DOPamine STD 15 mg Dextrose 5% 250 ml  Premix (mcg/kg/min)'
                 , 'DOPamine STD 400 mg Dextrose 5% 250 ml  Premix (mcg/kg/min)'
                 , 'DOPamine STD 400 mg Dextrose 5% 500 ml  Premix (mcg/kg/min)'
                 , 'Dopamine (Unknown)'
              )
              -- note: no rows found for inotropin
                then 1
              else null end
            ) as dopamine

    -- this like statement is pretty reliable - no false positives when I checked
    -- also catches the brand name dobutrex
    , max(case when lower(drugname) like '%dobu%' then 1 else null end) as dobutamine
    , max(case
              when drugname in
              (
                 'Norepinephrine'
               , 'Norepinephrine ()'
               , 'Norepinephrine MAX 32 mg Dextrose 5% 250 ml (mcg/min)'
               , 'Norepinephrine MAX 32 mg Dextrose 5% 500 ml (mcg/min)'
               , 'Norepinephrine (mcg/hr)'
               , 'Norepinephrine (mcg/kg/hr)'
               , 'Norepinephrine (mcg/kg/min)'
               , 'Norepinephrine (mcg/min)'
               , 'Norepinephrine (mg/hr)'
               , 'Norepinephrine (mg/kg/min)'
               , 'Norepinephrine (mg/min)'
               , 'Norepinephrine (ml/hr)'
               , 'Norepinephrine STD 32 mg Dextrose 5% 282 ml (mcg/min)'
               , 'Norepinephrine STD 32 mg Dextrose 5% 500 ml (mcg/min)'
               , 'Norepinephrine STD 4 mg Dextrose 5% 250 ml (mcg/min)'
               , 'Norepinephrine STD 4 mg Dextrose 5% 500 ml (mcg/min)'
               , 'Norepinephrine STD 8 mg Dextrose 5% 250 ml (mcg/min)'
               , 'Norepinephrine STD 8 mg Dextrose 5% 500 ml (mcg/min)'
               , 'Norepinephrine (units/min)'
               , 'Norepinephrine (Unknown)'
               , 'norepinephrine Volume (ml)'
               , 'norepinephrine Volume (ml) (ml/hr)'
               -- levophed
              , 'Levophed (mcg/kg/min)'
              , 'levophed  (mcg/min)'
              , 'levophed (mcg/min)'
              , 'Levophed (mcg/min)'
              , 'Levophed (mg/hr)'
              , 'levophed (ml/hr)'
              , 'Levophed (ml/hr)'
              , 'NSS with LEVO (ml/hr)'
              , 'NSS w/ levo/vaso (ml/hr)'
              )
          then 1 else 0 end) as norepinephrine
    , max(case
          when drugname in
          (
             'Phenylephrine'
           , 'Phenylephrine ()'
           , 'Phenylephrine  MAX 100 mg Sodium Chloride 0.9% 250 ml (mcg/min)'
           , 'Phenylephrine (mcg/hr)'
           , 'Phenylephrine (mcg/kg/min)'
           , 'Phenylephrine (mcg/kg/min) (mcg/kg/min)'
           , 'Phenylephrine (mcg/min)'
           , 'Phenylephrine (mcg/min) (mcg/min)'
           , 'Phenylephrine (mg/hr)'
           , 'Phenylephrine (mg/kg/min)'
           , 'Phenylephrine (ml/hr)'
           , 'Phenylephrine  STD 20 mg Sodium Chloride 0.9% 250 ml (mcg/min)'
           , 'Phenylephrine  STD 20 mg Sodium Chloride 0.9% 500 ml (mcg/min)'
           , 'Volume (ml) Phenylephrine'
           , 'Volume (ml) Phenylephrine ()'
           -- neosynephrine is a synonym
           , 'neo-synephrine (mcg/min)'
           , 'neosynephrine (mcg/min)'
           , 'Neosynephrine (mcg/min)'
           , 'Neo Synephrine (mcg/min)'
           , 'Neo-Synephrine (mcg/min)'
           , 'NeoSynephrine (mcg/min)'
           , 'NEO-SYNEPHRINE (mcg/min)'
           , 'Neosynephrine (ml/hr)'
           , 'neosynsprine'
           , 'neosynsprine (mcg/kg/hr)'
          )
        then 1 else 0 end) as phenylephrine
    , max(case
            when drugname in
            (
                 'EPI (mcg/min)'
               , 'Epinepherine (mcg/min)'
               , 'Epinephrine'
               , 'Epinephrine ()'
               , 'EPINEPHrine(Adrenalin)MAX 30 mg Sodium Chloride 0.9% 250 ml (mcg/min)'
               , 'EPINEPHrine(Adrenalin)STD 4 mg Sodium Chloride 0.9% 250 ml (mcg/min)'
               , 'EPINEPHrine(Adrenalin)STD 4 mg Sodium Chloride 0.9% 500 ml (mcg/min)'
               , 'EPINEPHrine(Adrenalin)STD 7 mg Sodium Chloride 0.9% 250 ml (mcg/min)'
               , 'Epinephrine (mcg/hr)'
               , 'Epinephrine (mcg/kg/min)'
               , 'Epinephrine (mcg/min)'
               , 'Epinephrine (mg/hr)'
               , 'Epinephrine (mg/kg/min)'
               , 'Epinephrine (ml/hr)'
            ) then 1 else 0 end)
          as epinephrine
    , max(case
            when drugname in
            (
                'Vasopressin'
              , 'Vasopressin ()'
              , 'Vasopressin 20 Units Sodium Chloride 0.9% 100 ml (units/hr)'
              , 'Vasopressin 20 Units Sodium Chloride 0.9% 250 ml (units/hr)'
              , 'Vasopressin 40 Units Sodium Chloride 0.9% 100 ml (units/hr)'
              , 'Vasopressin 40 Units Sodium Chloride 0.9% 100 ml (units/kg/hr)'
              , 'Vasopressin 40 Units Sodium Chloride 0.9% 100 ml (units/min)'
              , 'Vasopressin 40 Units Sodium Chloride 0.9% 100 ml (Unknown)'
              , 'Vasopressin 40 Units Sodium Chloride 0.9% 200 ml (units/min)'
              , 'Vasopressin (mcg/kg/min)'
              , 'Vasopressin (mcg/min)'
              , 'Vasopressin (mg/hr)'
              , 'Vasopressin (mg/min)'
              , 'vasopressin (ml/hr)'
              , 'Vasopressin (ml/hr)'
              , 'Vasopressin (units/hr)'
              , 'Vasopressin (units/kg/min)'
              , 'vasopressin (units/min)'
              , 'Vasopressin (units/min)'
              , 'VAsopressin (units/min)'
              , 'Vasopressin (Unknown)'
            ) then 1 else 0 end)
          as vasopressin
    , max(case when drugname in
      (
           'Milrinone'
         , 'Milrinone ()'
         , 'Milrinone (mcg/kg/hr)'
         , 'Milrinone (mcg/kg/min)'
         , 'Milrinone (ml/hr)'
         , 'Milrinone (Primacor) 40 mg Dextrose 5% 200 ml (mcg/kg/min)'
         , 'Milronone (mcg/kg/min)'
         , 'primacore (mcg/kg/min)'
      ) then 1 else 0 end)
      as milrinone
    , max(case when drugname in
      (
          'Hepain (ml/hr)'
        , 'Heparin'
        , 'Heparin ()'
        , 'Heparin 25,000 Unit/D5w 250 ml (ml/hr)'
        , 'Heparin 25000 Units Dextrose 5% 500 ml  Premix (units/hr)'
        , 'Heparin 25000 Units Dextrose 5% 500 ml  Premix (units/kg/hr)'
        , 'Heparin 25000 Units Dextrose 5% 950 ml  Premix (units/kg/hr)'
        , 'HEPARIN #2 (units/hr)'
        , 'Heparin 8000u/1L NS (ml/hr)'
        , 'Heparin-EKOS (units/hr)'
        , 'Heparin/Femoral Sheath   (units/hr)'
        , 'Heparin (mcg/kg/hr)'
        , 'Heparin (mcg/kg/min)'
        , 'Heparin (ml/hr)'
        , 'heparin (units/hr)'
        , 'Heparin (units/hr)'
        , 'HEPARIN (units/hr)'
        , 'Heparin (units/kg/hr)'
        , 'Heparin (Unknown)'
        , 'Heparin via sheath (units/hr)'
        , 'Left  Heparin (units/hr)'
        , 'NSS carrier heparin (ml/hr)'
        , 'S-Heparin (units/hr)'
        , 'Volume (ml) Heparin-heparin 25,000 units in 0.45 % sodium chloride 500 mL infusion'
        , 'Volume (ml) Heparin-heparin 25,000 units in 0.45 % sodium chloride 500 mL infusion (ml/hr)'
        , 'Volume (ml) Heparin-heparin 25,000 units in dextrose 500 mL infusion'
        , 'Volume (ml) Heparin-heparin 25,000 units in dextrose 500 mL infusion (ml/hr)'
        , 'Volume (ml) Heparin-heparin infusion 2 units/mL in 0.9% sodium chloride (ARTERIAL LINE)'
        , 'Volume (ml) Heparin-heparin infusion 2 units/mL in 0.9% sodium chloride (ARTERIAL LINE) (ml/hr)'
      ) then 1 else 0 end)
      as heparin
  from eicu.infusiondrug
  group by patientunitstayid, infusionoffset
),

vasopressin_raw as(
select
  patientunitstayid
  , infusionoffset as chartoffset
  , dopamine::SMALLINT as dopamine
  , dobutamine::SMALLINT as dobutamine
  , norepinephrine::SMALLINT as norepinephrine
  , phenylephrine::SMALLINT as phenylephrine
  , epinephrine::SMALLINT as epinephrine
  , vasopressin::SMALLINT as vasopressin
  , milrinone::SMALLINT as milrinone
  , heparin::SMALLINT as heparin
from vw0
-- at least one of our drugs should be non-zero
where dopamine = 1
OR dobutamine = 1
OR norepinephrine = 1
OR phenylephrine = 1
OR epinephrine = 1
OR vasopressin = 1
OR milrinone = 1
OR heparin = 1
order by patientunitstayid, infusionoffset
),

vasopressin as (
select vasopressin_raw.patientunitstayid,
max(vasopressin) as vasopressin,
max(heparin) as heparin
from vasopressin_raw
group by vasopressin_raw.patientunitstayid
order by vasopressin_raw.patientunitstayid
)


select de.patientunitstayid, de.age, de.gender, de.ethnicity, de.unitType, de.admissionHeight, de.admissionweight, de.dischargeweight
--, de.apacheScore, de.apacheVersion
, de.ICU_mort, de.ICU_los, de.hosp_mort, de.hosp_los, de.ventilation_duration_days
, fluid.nettotal, fluid.intakeOutputOffset as fluid_offset_min
, history.Hypertension, history.Atrial_Fibrillation, history.Cancer, history.CHF, history.CKD, history.CLD
, history.COPD , history.Diabetes, history.IHD, history.Cebrovascular_Accident
, sepsis_data.sepsis_binary
, vasopressor.vasopressor
, vaso_infusion.vasopressin, vaso_infusion.heparin
, AKI_creat.aki as aki_creat
, icustay_detail.apache_iv

from demographics de
left join fluid
on de.patientunitstayid = fluid.patientunitstayid
left join history
on de.patientUnitStayID = history.patientUnitStayID
left join sepsis_data
on de.patientUnitStayID = sepsis_data.patientUnitStayID
left join vasopressor
on de.patientUnitStayID = vasopressor.patientUnitStayID
left join vasopressin vaso_infusion
on de.patientUnitStayID = vaso_infusion.patientUnitStayID
left join BP_project.AKI_creat
on de.patientUnitStayID = AKI_creat.patientUnitStayID
left join public.icustay_detail
on de.patientUnitStayID = icustay_detail.patientUnitStayID;



"""

BP_data_table1 = pd.read_sql_query(query1,con)
BP_data_table1.head()


,patientunitstayid,age,gender,ethnicity,unittype,admissionheight,admissionweight,dischargeweight,icu_mort,icu_los,...,copd,diabetes,ihd,cebrovascular_accident,sepsis_binary,vasopressor,vasopressin,heparin,aki_creat,apache_iv
0,141168,70,Female,Caucasian,Med-Surg ICU,152.4,84.3,85.8,EXPIRED,2.4972,...,1.0,0.0,1.0,0.0,0,NaN,NaN,NaN,1.0,65.0
1,141194,68,Male,Caucasian,CTICU,180.3,73.9,76.7,ALIVE,3.3423,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,0.0,70.0
2,141203,77,Female,Caucasian,Med-Surg ICU,160.0,70.2,70.3,ALIVE,1.2979,...,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,1.0,90.0
3,141229,> 89,Female,Caucasian,Med-Surg ICU,160.0,89.8,86.6,None,NaN,...,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,0.0,NaN
4,141233,81,Female,Caucasian,CTICU,165.1,61.7,73.2,ALIVE,10.8923,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,66.0


In [213]:
BP_data_table1.shape # (21348, 31)
#BP_data_table1.shape (22518, 31)

(21338, 31)

In [214]:
BP_data_table1.to_csv("BP_project/BP_data_table1_creatAKI.csv")

# Redefine AKI based on both creatinine and Urine output

In [37]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password='eicu')
cur = con.cursor()
cur.execute('SET search_path to ' + 'public')

query = \
"""
select distinct (patientunitstayid) from pivoted_uo;
"""

data = pd.read_sql_query(query,con)
data.head()

,patientunitstayid
0,233338
1,2880280
2,2673681
3,522410
4,2305170


In [53]:
data.shape

(158442, 1)

# Define AKI based on 7days urinne output

In [43]:
df = pd.read_csv("./time_series.csv", engine = 'python')
df.head()

,Time,intime,patientunitstayid,Weight,chartoffset,6 hours urine output,AKI
0,2014-01-01 04:00:00,2014-01-01 03:43:00,224606,0.0,17.0,0.0,0
1,2014-01-01 05:00:00,2014-01-01 03:43:00,224606,0.0,77.0,0.0,0
2,2014-01-01 06:00:00,2014-01-01 03:43:00,224606,0.0,137.0,0.0,0
3,2014-01-01 07:00:00,2014-01-01 03:43:00,224606,0.0,197.0,0.0,0
4,2014-01-01 08:00:00,2014-01-01 03:43:00,224606,0.0,257.0,0.0,0


In [70]:
df_7day = df[(df['chartoffset']>=360) & (df['chartoffset']<=10080)]

In [71]:
df_7day.head(2)

,Time,intime,patientunitstayid,Weight,chartoffset,6 hours urine output,AKI
6,2014-01-01 10:00:00,2014-01-01 03:43:00,224606,0.0,377.0,0.0,0
7,2014-01-01 11:00:00,2014-01-01 03:43:00,224606,0.0,437.0,0.0,0


In [72]:
df_7day.patientunitstayid.unique().shape

(23928,)

In [73]:
df_new = df_7day.loc[df['patientunitstayid'].isin(data['patientunitstayid'])]

In [74]:
df_new.patientunitstayid.unique().shape

(21727,)

In [75]:
grouped_df = df_new.groupby("patientunitstayid")
maximums = grouped_df["AKI"].max()
maximums = maximums.reset_index()

In [76]:
maximums.head()

,patientunitstayid,AKI
0,141179,0
1,141194,1
2,141233,1
3,141244,1
4,141266,1


In [77]:
maximums.columns = ['patientunitstayid', 'AKI_uo7day']
maximums.head(2)

,patientunitstayid,AKI_uo7day
0,141179,0
1,141194,1


In [78]:
maximums.to_csv("./Analysis/csv_folder/eicu_AKI_uo7day.csv")

# AKI within ICU stay

In [222]:
df = pd.read_csv("BP_project/BP_data_table1_creatAKI.csv", engine = 'python')
df.head()

,Unnamed: 0,patientunitstayid,age,gender,ethnicity,unittype,admissionheight,admissionweight,dischargeweight,icu_mort,...,copd,diabetes,ihd,cebrovascular_accident,sepsis_binary,vasopressor,vasopressin,heparin,aki_creat,apache_iv
0,0,141168,70,Female,Caucasian,Med-Surg ICU,152.4,84.3,85.8,EXPIRED,...,1.0,0.0,1.0,0.0,0,NaN,NaN,NaN,1.0,65.0
1,1,141194,68,Male,Caucasian,CTICU,180.3,73.9,76.7,ALIVE,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,0.0,70.0
2,2,141203,77,Female,Caucasian,Med-Surg ICU,160.0,70.2,70.3,ALIVE,...,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,1.0,90.0
3,3,141229,> 89,Female,Caucasian,Med-Surg ICU,160.0,89.8,86.6,NaN,...,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,0.0,NaN
4,4,141233,81,Female,Caucasian,CTICU,165.1,61.7,73.2,ALIVE,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,66.0


In [223]:
df_all = pd.merge(df, maximums.rename(columns={'patientunitstayid':'patientunitstayid'}), on='patientunitstayid',  how='left')
df_all.rename({'AKI': 'aki_uo'}, axis=1, inplace=True)
df_all.head(2)

,Unnamed: 0,patientunitstayid,age,gender,ethnicity,unittype,admissionheight,admissionweight,dischargeweight,icu_mort,...,diabetes,ihd,cebrovascular_accident,sepsis_binary,vasopressor,vasopressin,heparin,aki_creat,apache_iv,aki_uo
0,0,141168,70,Female,Caucasian,Med-Surg ICU,152.4,84.3,85.8,EXPIRED,...,0.0,1.0,0.0,0,NaN,NaN,NaN,1.0,65.0,1
1,1,141194,68,Male,Caucasian,CTICU,180.3,73.9,76.7,ALIVE,...,NaN,NaN,NaN,1,NaN,NaN,NaN,0.0,70.0,1


In [224]:
df_all["AKI"] = df_all[["aki_creat", "aki_uo"]].max(axis=1)

In [225]:
df_all.to_csv("BP project/BP_data_table1_new.csv")

# SOFA score

In [ ]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password='eicus')
cur = con.cursor()
cur.execute('SET search_path to ' + 'public')

query3 = \
"""
--IMPORTANT: Please see around l277 for some possible issues.
--
--Based on code by Matthieu Komorowski, with changes to allow it to be used 
-- on BQ

WITH 

pat AS (
SELECT * FROM `oxygenators-209612.eicu.patient`),

lab AS (
SELECT * FROM `oxygenators-209612.eicu.lab`),

vitalperiodic AS (
SELECT * FROM `oxygenators-209612.eicu.vitalperiodic`),

vitalaperiodic AS (
SELECT * FROM `oxygenators-209612.eicu.vitalaperiodic`),

infusiondrug AS (
SELECT * FROM `oxygenators-209612.eicu.infusiondrug`),

respiratorycare AS (
SELECT * FROM `oxygenators-209612.eicu.respiratorycare`),

treatment AS (
SELECT * FROM `oxygenators-209612.eicu.treatment`),

careplangeneral AS (
SELECT * FROM `oxygenators-209612.eicu.careplangeneral`),

physicalexam AS (
SELECT * FROM `oxygenators-209612.eicu.physicalexam`),

diag AS (
SELECT * FROM `oxygenators-209612.eicu.diagnosis`),

chart AS (
SELECT * FROM `oxygenators-209612.eicu.nursecharting`),

apsiii_raw AS (
SELECT * FROM `oxygenators-209612.eicu.apachepatientresult`),

intakeoutput AS (
SELECT * FROM `oxygenators-209612.eicu.intakeoutput`),

respchart AS (
SELECT * FROM `oxygenators-209612.eicu.respiratorycharting`),


cohort1 AS (
SELECT * FROM `oxygenators-209612.eicu.patient`),


t1 as -- MAP
(
WITH tt1 as
(
select patientunitstayid,
min( case when noninvasivemean is not null then noninvasivemean else null end) as map
from vitalaperiodic
where observationoffset between -1440 and 1440
group by patientunitstayid
), 

tt2 as
(
select patientunitstayid,
min( case when systemicmean is not null then systemicmean else null end) as map
from vitalperiodic
where observationoffset between -1440 and 1440
group by patientunitstayid
)


select pt.patientunitstayid, case when tt1.map is not null then tt1.map
when tt2.map is not null then tt2.map
else null end as map
from pat pt
left outer join tt1
on tt1.patientunitstayid=pt.patientunitstayid
left outer join tt2
on tt2.patientunitstayid=pt.patientunitstayid
order by pt.patientunitstayid
),

t2 as --DOPAMINE
(
select distinct  patientunitstayid, max(
case when lower(drugname) like '%(ml/hr)%' then round(cast(drugrate as numeric)/3,3) -- rate in ml/h * 1600 mcg/ml / 80 kg / 60 min, to convert in mcg/kg/min
when lower(drugname) like '%(mcg/kg/min)%' then cast(drugrate as numeric)
else null end ) as dopa
from infusiondrug id
where lower(drugname) like '%dopamine%' and infusionoffset between -120 and 1440 and REGEXP_CONTAINS(drugrate, '^[0-9]{0,5}$') and drugrate<>'' and drugrate<>'.'
group by patientunitstayid
order by patientunitstayid


), 

t3 as  --NOREPI
(
select distinct patientunitstayid, max(case when lower(drugname) like '%(ml/hr)%' and drugrate<>''  and drugrate<>'.' then round(cast(drugrate as numeric)/300,3) -- rate in ml/h * 16 mcg/ml / 80 kg / 60 min, to convert in mcg/kg/min
when lower(drugname) like '%(mcg/min)%' and drugrate<>'' and drugrate<>'.'  then round(cast(drugrate as numeric)/80 ,3)-- divide by 80 kg
when lower(drugname) like '%(mcg/kg/min)%' and drugrate<>'' and drugrate<>'.' then cast(drugrate as numeric)
else null end ) as norepi


from infusiondrug id
where lower(drugname) like '%norepinephrine%'  and infusionoffset between -120 and 1440  and REGEXP_CONTAINS(drugrate, '^[0-9]{0,5}$') and drugrate<>'' and drugrate<>'.'
group by patientunitstayid
order by patientunitstayid


), 

t4 as  --DOBUTAMINE
(
select distinct patientunitstayid, 1 as dobu
from infusiondrug id
where lower(drugname) like '%dobutamin%' and drugrate <>'' and drugrate<>'.' and drugrate <>'0' and REGEXP_CONTAINS(drugrate, '^[0-9]{0,5}$') and infusionoffset between -120 and 1440
order by patientunitstayid
),

sofacv as
(
select pt.patientunitstayid, t1.map, t2.dopa, t3.norepi, t4.dobu,
(case when dopa>=15 or norepi>0.1 then 4
when dopa>5 or (norepi>0 and norepi <=0.1) then 3
when dopa<=5 or dobu > 0 then 2
when map <70 then 1
else 0 end) as SOFA_cv --COMPUTE SOFA CV
from cohort1 pt
left outer join t1
on t1.patientunitstayid=pt.patientunitstayid
left outer join t2
on t2.patientunitstayid=pt.patientunitstayid
left outer join t3
on t3.patientunitstayid=pt.patientunitstayid
left outer join t4
on t4.patientunitstayid=pt.patientunitstayid
order by pt.patientunitstayid
),


-- SOFA-RESPI


sofarespi as
(
with tempo2 as 
(
with tempo1 as
(
with t1 as --FIO2 from respchart
(
select *
from
(
select distinct patientunitstayid, max(cast(respchartvalue as numeric)) as rcfio2
-- , max(case when respchartvaluelabel = 'FiO2' then respchartvalue else null end) as fiO2
from respchart
where respchartoffset between -120 and 1440 and respchartvalue <> '' and REGEXP_CONTAINS(respchartvalue, '^[0-9]{0,2}$')
group by patientunitstayid
) as tempo
where rcfio2 >20 -- many values are liters per minute!
order by patientunitstayid


), 

t2 as --FIO2 from nursecharting
(
select distinct patientunitstayid, max(cast(nursingchartvalue as numeric)) as ncfio2
from chart nc
where lower(nursingchartcelltypevallabel) like '%fio2%' and REGEXP_CONTAINS(nursingchartvalue, '^[0-9]{0,2}$') and nursingchartentryoffset between -120 and 1440
group by patientunitstayid


), 

t3 as --sao2 from vitalperiodic
(
select patientunitstayid,
min( case when sao2 is not null then sao2 else null end) as sao2
from vitalperiodic
where observationoffset between -1440 and 1440
group by patientunitstayid


), 

t4 as --pao2 from lab
(
select patientunitstayid,
min(case when lower(labname) like 'pao2%' then labresult else null end) as pao2
from lab
where labresultoffset between -1440 and 1440
group by patientunitstayid


), 

t5 as --airway type combining 3 sources (1=invasive)
(


with t1 as --airway type from respcare (1=invasive) (by resp therapist!!)
(
select distinct patientunitstayid,
max(case when airwaytype in ('Oral ETT','Nasal ETT','Tracheostomy') then 1 else NULL end) as airway  -- either invasive airway or NULL
from respiratorycare
where respcarestatusoffset between -1440 and 1440


group by patientunitstayid-- , respcarestatusoffset
-- order by patientunitstayid-- , respcarestatusoffset
),


t2 as --airway type from respcharting (1=invasive)
(
select distinct patientunitstayid, 1 as ventilator
from respchart rc
where respchartvalue like '%ventilator%'
or respchartvalue like '%vent%'
or respchartvalue like '%bipap%'
or respchartvalue like '%840%'
or respchartvalue like '%cpap%'
or respchartvalue like '%drager%'
or respchartvalue like 'mv%'
or respchartvalue like '%servo%'
or respchartvalue like '%peep%'
and respchartoffset between -1440 and 1440
group by patientunitstayid
-- order by patientunitstayid
),


t3 as --airway type from treatment (1=invasive)


(
select distinct patientunitstayid, max(case when treatmentstring in
('pulmonary|ventilation and oxygenation|mechanical ventilation',
'pulmonary|ventilation and oxygenation|tracheal suctioning',
'pulmonary|ventilation and oxygenation|ventilator weaning',
'pulmonary|ventilation and oxygenation|mechanical ventilation|assist controlled',
'pulmonary|radiologic procedures / bronchoscopy|endotracheal tube',
'pulmonary|ventilation and oxygenation|oxygen therapy (> 60%)',
'pulmonary|ventilation and oxygenation|mechanical ventilation|tidal volume 6-10 ml/kg',
'pulmonary|ventilation and oxygenation|mechanical ventilation|volume controlled',
'surgery|pulmonary therapies|mechanical ventilation',
'pulmonary|surgery / incision and drainage of thorax|tracheostomy',
'pulmonary|ventilation and oxygenation|mechanical ventilation|synchronized intermittent',
'pulmonary|surgery / incision and drainage of thorax|tracheostomy|performed during current admission for ventilatory support',
'pulmonary|ventilation and oxygenation|ventilator weaning|active',
'pulmonary|ventilation and oxygenation|mechanical ventilation|pressure controlled',
'pulmonary|ventilation and oxygenation|mechanical ventilation|pressure support',
'pulmonary|ventilation and oxygenation|ventilator weaning|slow',
'surgery|pulmonary therapies|ventilator weaning',
'surgery|pulmonary therapies|tracheal suctioning',
'pulmonary|radiologic procedures / bronchoscopy|reintubation',
'pulmonary|ventilation and oxygenation|lung recruitment maneuver',
'pulmonary|surgery / incision and drainage of thorax|tracheostomy|planned',
'surgery|pulmonary therapies|ventilator weaning|rapid',
'pulmonary|ventilation and oxygenation|prone position',
'pulmonary|surgery / incision and drainage of thorax|tracheostomy|conventional',
'pulmonary|ventilation and oxygenation|mechanical ventilation|permissive hypercapnea',
'surgery|pulmonary therapies|mechanical ventilation|synchronized intermittent',
'pulmonary|medications|neuromuscular blocking agent',
'surgery|pulmonary therapies|mechanical ventilation|assist controlled',
'pulmonary|ventilation and oxygenation|mechanical ventilation|volume assured',
'surgery|pulmonary therapies|mechanical ventilation|tidal volume 6-10 ml/kg',
'surgery|pulmonary therapies|mechanical ventilation|pressure support',
'pulmonary|ventilation and oxygenation|non-invasive ventilation',
'pulmonary|ventilation and oxygenation|non-invasive ventilation|face mask',
'pulmonary|ventilation and oxygenation|non-invasive ventilation|nasal mask',
'pulmonary|ventilation and oxygenation|mechanical ventilation|non-invasive ventilation',
'pulmonary|ventilation and oxygenation|mechanical ventilation|non-invasive ventilation|face mask',
'surgery|pulmonary therapies|non-invasive ventilation',
'surgery|pulmonary therapies|non-invasive ventilation|face mask',
'pulmonary|ventilation and oxygenation|mechanical ventilation|non-invasive ventilation|nasal mask',
'surgery|pulmonary therapies|non-invasive ventilation|nasal mask',
'surgery|pulmonary therapies|mechanical ventilation|non-invasive ventilation',
'surgery|pulmonary therapies|mechanical ventilation|non-invasive ventilation|face mask'
) then 1  else NULL end) as interface   -- either ETT/NiV or NULL
from treatment
where treatmentoffset between -1440 and 1440
group by patientunitstayid-- , treatmentoffset, interface
order by patientunitstayid-- , treatmentoffset
),

t4 as
(
select distinct patientunitstayid,
max(case when cplitemvalue like '%Intubated%' then 1 else NULL end) as airway  -- either invasive airway or NULL
from careplangeneral
where cplitemoffset between -1440 and 1440
group by patientunitstayid -- , respcarestatusoffset


)

--Note from Michael
--
--Previously the below line was "case when t1.airway is not null or t2.ventilator is not null or t3.interface is not null or t4.interface is not null then 1 else null end as mechvent
--
--t4 doesn't have interface, removing

select pt.patientunitstayid,
case when t1.airway is not null or t2.ventilator is not null or t3.interface is not null then 1 else null end as mechvent --summarize
from cohort1 pt
left outer join t1
on t1.patientunitstayid=pt.patientunitstayid
left outer join t2
on t2.patientunitstayid=pt.patientunitstayid
left outer join t3
on t3.patientunitstayid=pt.patientunitstayid
left outer join t4
on t4.patientunitstayid=pt.patientunitstayid

--Note from Michael
--
--Previously the last line was "on t4.patientunitstayid=pt.patientunitstayidorder by pt.patientunitstayid"
--
--No idea what this is. "patientunitstayidorder site:eicu-crd.mit.edu" has no hits.


)


select pt.patientunitstayid, t3.sao2, t4.pao2, 
(case when t1.rcfio2>20 then t1.rcfio2 when t2.ncfio2 >20 then t2.ncfio2  when t1.rcfio2=1 or t2.ncfio2=1 then 100 else null end) as fio2, t5.mechvent
from cohort1 pt
left outer join t1
on t1.patientunitstayid=pt.patientunitstayid
left outer join t2
on t2.patientunitstayid=pt.patientunitstayid
left outer join t3
on t3.patientunitstayid=pt.patientunitstayid
left outer join t4
on t4.patientunitstayid=pt.patientunitstayid
left outer join t5
on t5.patientunitstayid=pt.patientunitstayid
-- order by pt.patientunitstayid
)


select *, -- coalesce(fio2,nullif(fio2,0),21) as fn, nullif(fio2,0) as nullifzero, coalesce(coalesce(nullif(fio2,0),21),fio2,21) as ifzero21 ,
coalesce(pao2,100)/coalesce(coalesce(nullif(fio2,0),21),fio2,21) as pf, coalesce(sao2,100)/coalesce(coalesce(nullif(fio2,0),21),fio2,21) as sf
from tempo1
)


select patientunitstayid, 
(case when pf <1 or sf <0.67 then 4  --COMPUTE SOFA RESPI
when pf between 1 and 2 or sf between 0.67 and 1.41 then 3
when pf between 2 and 3 or sf between 1.42 and 2.2 then 2
when pf between 3 and 4 or sf between 2.21 and 3.01 then 1
when pf > 4 or sf> 3.01 then 0 else 0 end ) as SOFA_respi
from tempo2
order by patientunitstayid
),


-- SOFA-RENAL


sofarenal as
(
with t1 as --CREATININE
(
select pt.patientunitstayid,
max(case when lower(labname) like 'creatin%' then labresult else null end) as creat
from pat pt
left outer join lab
on pt.patientunitstayid=lab.patientunitstayid
where labresultoffset between -1440 and 1440
group by pt.patientunitstayid


),

t2 as --UO
(


with uotemp as
(
select patientunitstayid,
case when dayz=1 then sum(outputtotal) else null end as uod1
from
(


select distinct patientunitstayid, intakeoutputoffset, outputtotal,
(CASE
WHEN  (intakeoutputoffset) between -120 and 1440 THEN 1
else null
end) as dayz
from intakeoutput
where intakeoutputoffset between 0 and 5760
order by patientunitstayid, intakeoutputoffset


) as temp
group by patientunitstayid, temp.dayz
)


select pt.patientunitstayid,
max(case when uod1 is not null then uod1 else null end) as UO
from pat pt
left outer join uotemp
on uotemp.patientunitstayid=pt.patientunitstayid
group by pt.patientunitstayid


)


select pt.patientunitstayid, -- t1.creat, t2.uo,
(case --COMPUTE SOFA RENAL
when uo <200 or creat>5 then 4
when uo <500 or creat >3.5 then 3
when creat between 2 and 3.5 then 2
when creat between 1.2 and 2 then 1
else 0
end) as sofarenal
from cohort1 pt
left outer join t1
on t1.patientunitstayid=pt.patientunitstayid
left outer join t2
on t2.patientunitstayid=pt.patientunitstayid
order by pt.patientunitstayid
-- group by pt.patientunitstayid, t1.creat, t2.uo


),


-- SOFA- GCS, liver, platelets


sofa3others as
(
with t1 as --GCS
(
select patientunitstayid, sum(cast(physicalexamvalue as numeric)) as gcs
from physicalexam pe
where (lower(physicalexampath) like '%gcs/eyes%'
or lower(physicalexampath) like '%gcs/verbal%'
or lower(physicalexampath) like '%gcs/motor%')
and physicalexamoffset between -1440 and 1440
group by patientunitstayid--, physicalexamoffset
), t2 as
(
select pt.patientunitstayid,
max(case when lower(labname) like 'total bili%' then labresult else null end) as bili, --BILI
min(case when lower(labname) like 'platelet%' then labresult else null end) as plt --PLATELETS
from pat pt
left outer join lab
on pt.patientunitstayid=lab.patientunitstayid
where labresultoffset between -1440 and 1440
group by pt.patientunitstayid
)


select distinct pt.patientunitstayid, min(t1.gcs) as gcs, max(t2.bili) as bili, min(t2.plt) as plt,
max(case when plt<20 then 4
when plt<50 then 3
when plt<100 then 2
when plt<150 then 1
else 0 end) as sofacoag,
max(case when bili>12 then 4
when bili>6 then 3
when bili>2 then 2
when bili>1.2 then 1
else 0 end) as sofaliver,
max(case when gcs=15 then 0
when gcs>=13 then 1
when gcs>=10 then 2
when gcs>=6 then 3
when gcs>=3 then 4
else 0 end) as sofacns
from cohort1 pt
left outer join t1
on t1.patientunitstayid=pt.patientunitstayid
left outer join t2
on t2.patientunitstayid=pt.patientunitstayid
group by pt.patientunitstayid, t1.gcs, t2.bili, t2.plt
order by pt.patientunitstayid
)


-- SOFA: COMBINE ALL SUBSCORES 


Select pt.patientunitstayid, --  sofacv.sofa_cv, sofarespi.sofa_respi,sofarenal.sofarenal,sofa3others.sofacoag,sofa3others.sofaliver,sofa3others.sofacns, 
sofacv.sofa_cv+sofarespi.sofa_respi+ sofarenal.sofarenal+sofa3others.sofacoag+ sofa3others.sofaliver+sofa3others.sofacns as sofatotal
From cohort1 pt
Left outer join sofacv
On pt.patientunitstayid=sofacv.Patientunitstayid
Left outer join sofarespi
On pt.patientunitstayid= sofarespi.Patientunitstayid
Left outer join sofarenal
On pt.patientunitstayid= sofarenal.Patientunitstayid
Left outer join sofa3others
On pt.patientunitstayid= sofa3others.Patientunitstayid



"""

BP_sofa = pd.read_sql_query(query3,con)
BP_sofa.head()


# BP data table 2

In [211]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password='eicus')
cur = con.cursor()
cur.execute('SET search_path to ' + 'public')

query2 = \
"""
with firstIcu as (
     SELECT  uniquepid
                , patienthealthsystemstayid as hadmid
                , patientunitstayid
                , row_number() OVER (PARTITION BY uniquepid, patienthealthsystemstayid ORDER BY unitvisitnumber) AS first_icu
     FROM public.icustay_detail
),

-- ############ first ICU admission ############ 
cohort_first as (
SELECT  uniquepid, hadmid, patientunitstayid
FROM  firstIcu
    WHERE first_icu = 1
),
-- ############ BP reading in the first 24h and has fulfilled criteria 1,2,3 ############
cohort as (
select cohort_first.patientunitstayid
from cohort_first inner join BP_project.BP_cohort
on cohort_first.patientunitstayid = BP_cohort.patientunitstayid
)

select pivoted_vital.*
from cohort left join public.pivoted_vital
on cohort.patientunitstayid = pivoted_vital.patientunitstayid
inner join BP_project.BP_cohort
on cohort.patientunitstayid = BP_cohort.patientunitstayid
where pivoted_vital.chartoffset >= BP_cohort.min_offset
or pivoted_vital.chartoffset <= BP_cohort.max_offset
order by cohort.patientunitstayid, pivoted_vital.chartoffset;



"""

BP_data_table2 = pd.read_sql_query(query2,con)
BP_data_table2.head()


,patientunitstayid,chartoffset,entryoffset,heartrate,respiratoryrate,spo2,nibp_systolic,nibp_diastolic,nibp_mean,temperature,temperaturelocation,ibp_systolic,ibp_diastolic,ibp_mean
0,141168,6,6,140.0,NaN,NaN,82.0,59.0,NaN,NaN,None,NaN,NaN,67.0
1,141168,21,21,70.0,NaN,94.0,NaN,NaN,NaN,NaN,None,NaN,NaN,75.0
2,141168,36,36,70.0,NaN,94.0,105.0,59.0,NaN,NaN,None,NaN,NaN,75.0
3,141168,51,51,140.0,NaN,94.0,103.0,65.0,NaN,NaN,None,NaN,NaN,80.0
4,141168,66,66,140.0,NaN,92.0,99.0,68.0,NaN,NaN,None,NaN,NaN,80.0


In [216]:
BP_data_table2.shape # (5449408, 14)
#BP_data_table2.shape (5505413, 14)

(5446422, 14)

In [217]:
BP_data_table2['patientunitstayid'].unique().shape # (21349,)
#BP_data_table2['patientunitstayid'].unique().shape (22518,)

(21338,)

In [218]:
BP_data_table2.to_csv("BP project/BP_data_table2_new.csv")

# Height MIMIC

In [109]:
con = psycopg2.connect(dbname='mimic', user=sqluser, password='mimic')
cur = con.cursor()
cur.execute('SET search_path to ' + 'public')

query2 = \
"""
-- This query extracts heights for adult ICU patients.
-- It uses all information from the patient's first ICU day.
-- This is done for consistency with other queries - it's not necessarily needed.
-- Height is unlikely to change throughout a patient's stay.

-- ** Requires the echodata view, generated by concepts/echo-data.sql

-- staging table to ensure all heights are in centimeters
with ce0 as
(
    SELECT
      c.icustay_id
      , case
        -- convert inches to centimetres
          when itemid in (920, 1394, 4187, 3486)
              then valuenum * 2.54
            else valuenum
        end as Height
    FROM mimiciii.chartevents c
    inner join mimiciii.icustays ie
        on c.icustay_id = ie.icustay_id
        and c.charttime <= (ie.intime + INTERVAL '1 day')
        and c.charttime > (ie.intime+INTERVAL '1 day') -- some fuzziness for admit time
    WHERE c.valuenum IS NOT NULL
    AND c.itemid in (226730,920, 1394, 4187, 3486,3485,4188) -- height
    AND c.valuenum != 0
    -- exclude rows marked as error
    AND (c.error IS NULL OR c.error = 0)
)
, ce as
(
    SELECT
        icustay_id
        -- extract the median height from the chart to add robustness against outliers
        , AVG(height) as Height_chart
    from ce0
    where height > 100
    group by icustay_id
)
-- requires the echo-data.sql query to run
-- this adds heights from the free-text echo notes
, echo as
(
    select
        ec.subject_id
        -- all echo heights are in inches
        , 2.54*AVG(height) as Height_Echo
    from echodata ec
    inner join mimiciii.icustays ie
        on ec.subject_id = ie.subject_id
        and ec.charttime < (ie.intime+ INTERVAL '1 day')
    where height is not null
    and height*2.54 > 100
    group by ec.subject_id
)
select
    ie.icustay_id
    , coalesce(ce.Height_chart, ec.Height_Echo) as height

    -- components
    , ce.height_chart
    , ec.height_echo
FROM mimiciii.icustays ie

-- filter to only adults
inner join mimiciii.patients pat
    on ie.subject_id = pat.subject_id
    and ie.intime > (pat.dob+ INTERVAL '1 year')

left join ce
    on ie.icustay_id = ce.icustay_id

left join echo ec
    on ie.subject_id = ec.subject_id;



"""

df_height = pd.read_sql_query(query2,con)
df_height.head()


,icustay_id,height,height_chart,height_echo
0,294299,158.75,None,158.75
1,281559,152.40,None,152.40
2,204300,185.42,None,185.42
3,285183,172.72,None,172.72
4,291604,167.64,None,167.64


In [110]:
df_height.shape

(53432, 4)

In [116]:
df_height.to_csv("./Analysis/csv_folder/mimic_height.csv")

# Calculate BP 24, 48, 72 hour median

In [46]:
import pandas as pd

In [47]:
BP_data_table2 = pd.read_csv("BP_data_table2_new.csv", engine = 'python', index_col=0)

In [48]:
BP_data_table2.head(3)

,patientunitstayid,chartoffset,entryoffset,heartrate,respiratoryrate,spo2,nibp_systolic,nibp_diastolic,nibp_mean,temperature,temperaturelocation,ibp_systolic,ibp_diastolic,ibp_mean
0,141168,6,6,140.0,NaN,NaN,82.0,59.0,NaN,NaN,NaN,NaN,NaN,67.0
1,141168,21,21,70.0,NaN,94.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0
2,141168,36,36,70.0,NaN,94.0,105.0,59.0,NaN,NaN,NaN,NaN,NaN,75.0


In [49]:
BP_group = BP_data_table2.groupby("patientunitstayid")

In [50]:
def avg_col(df, col, time):
    first_day_chart = df[(df['chartoffset']>0) & (df['chartoffset']<time)]
    col_mean = first_day_chart[col].median()
    return col_mean

In [51]:
ibp_sys_mean_24 = BP_group.apply(lambda x: avg_col(x, col="ibp_systolic", time=1440))
ibp_dias_mean_24 = BP_group.apply(lambda x: avg_col(x, col="ibp_diastolic", time=1440))
ibp_mean_mean_24 = BP_group.apply(lambda x: avg_col(x, col="ibp_mean", time=1440))

/home/ephlius/.virtualenvs/3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1018: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [52]:
ibp_sys_mean_48 = BP_group.apply(lambda x: avg_col(x, col="ibp_systolic", time=2880))
ibp_dias_mean_48 = BP_group.apply(lambda x: avg_col(x, col="ibp_diastolic", time=2880))
ibp_mean_mean_48 = BP_group.apply(lambda x: avg_col(x, col="ibp_mean", time=2880))

/home/ephlius/.virtualenvs/3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1018: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [53]:
ibp_sys_mean_72 = BP_group.apply(lambda x: avg_col(x, col="ibp_systolic", time=4320))
ibp_dias_mean_72 = BP_group.apply(lambda x: avg_col(x, col="ibp_diastolic", time=4320))
ibp_mean_mean_72 = BP_group.apply(lambda x: avg_col(x, col="ibp_mean", time=4320))

/home/ephlius/.virtualenvs/3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1018: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [60]:
# compare with LiangYu's result eICU
Result = pd.read_csv("./Analysis/BP_data_table_processed_new.csv", engine = 'python', index_col=0)
Result[["patientunitstayid", "ibp_diastolic_24h"]].tail(10)
ibp_dias_mean_24.tail(10)

,patientunitstayid,ibp_diastolic_24h
21328,3352922,55.0
21329,3352979,93.0
21330,3353015,50.5
21331,3353077,73.0
21332,3353087,51.0
21333,3353144,61.5
21334,3353194,71.0
21335,3353197,55.0
21336,3353216,65.5
21337,3353251,58.0


In [78]:
# compare with LiangYu's result MIMIC
mimic_result = pd.read_csv("./Analysis/MIMIC_data_clean.csv", engine = 'python', index_col=0)
mimic_result[["icustay_id", "ibp_diastolic_24h"]].head(10)

,icustay_id,ibp_diastolic_24h
0,271147,47.0
1,214619,67.0
2,297937,59.0
3,203786,59.5
4,248622,69.0
5,248978,74.0
6,292735,63.0
7,236689,56.0
8,237256,49.0
9,219987,61.0


In [84]:
mimic_table2 = pd.read_csv("./Analysis/csv_folder/table2.csv", engine = 'python')
mimic_table2.head(2)

,icustay_id,charttime,heartrate,sysbp,diasbp,meanbp,resprate,tempc,spo2,glucose,intime,chartoffset
0,200003,2199-08-02 19:15:00,132.0,NaN,NaN,NaN,25.0,39.555554,97.0,NaN,2199-08-02 19:50:04,-35.066667
1,200003,2199-08-02 20:00:00,119.0,NaN,NaN,NaN,35.0,NaN,97.0,NaN,2199-08-02 19:50:04,9.933333


In [64]:
BP_group_mimic = mimic_table2.groupby("icustay_id")

In [65]:
mimic_dias_median_24 = BP_group_mimic.apply(lambda x: avg_col(x, col="diasbp", time=1440))

/home/ephlius/.virtualenvs/3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1018: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [96]:
mimic_dias_median_24.head(30)

icustay_id
200003    62.5
200007     NaN
200009    57.0
200012     NaN
200014    63.5
200017     NaN
200018     NaN
200019     NaN
200020     NaN
200021    66.0
200023     NaN
200025    60.0
200028    46.0
200029    40.0
200030    65.0
200031     NaN
200033     NaN
200035     NaN
200036     NaN
200037     NaN
200039    48.0
200041     NaN
200044     NaN
200046     NaN
200047     NaN
200052    59.0
200058     NaN
200059    66.0
200060     NaN
200061     NaN
dtype: float64

In [92]:
mimic_result.sort_values(by=['icustay_id'], inplace=True)

In [97]:
mimic_result.head(15)

,subject_id,hadm_id,icustay_id,gender,admittime,dischtime,los_hospital,age,ethnicity,admission_type,...,exclude_tempc,ibp_systolic_24h,ibp_systolic_48h,ibp_systolic_72h,ibp_diastolic_24h,ibp_diastolic_48h,ibp_diastolic_72h,ibp_mean_24h,ibp_mean_48h,ibp_mean_72h
8839,27513,163557,200003,M,2199-08-02 17:02:00,2199-08-22 19:00:00,20.0819,48.2960,WHITE,EMERGENCY,...,False,110.0,106.0,108.0,62.5,60.5,62.0,80.0,77.00,78.0
4325,29904,129607,200009,F,2189-11-30 10:45:00,2189-12-06 15:00:00,6.1771,47.5560,WHITE,ELECTIVE,...,False,96.0,106.5,106.5,57.0,63.5,63.5,69.0,76.00,76.0
4241,9514,127229,200014,M,2105-02-16 23:15:00,2105-02-21 13:46:00,4.6049,84.7300,NaN,EMERGENCY,...,False,126.5,141.0,141.0,63.5,66.0,66.0,84.5,95.00,95.0
8867,19167,164161,200025,M,2113-08-25 09:28:00,2113-08-31 19:20:00,6.4111,49.4313,WHITE,EMERGENCY,...,False,106.0,104.5,104.5,60.0,61.0,61.0,74.0,75.00,75.0
2705,41710,181955,200028,M,2133-10-29 10:00:00,2133-11-01 14:54:00,3.2042,64.8677,WHITE,ELECTIVE,...,False,112.0,115.0,115.0,46.0,47.0,47.0,64.0,65.00,65.0
5534,14987,165426,200030,M,2150-11-13 14:06:00,2150-12-04 17:55:00,21.1590,54.1931,BLACK/AFRICAN AMERICAN,EMERGENCY,...,False,123.0,126.5,131.0,65.0,67.0,71.0,82.0,86.50,89.0
11249,20147,135441,200039,F,2121-12-26 03:24:00,2122-01-07 09:45:00,12.2646,69.2750,WHITE,EMERGENCY,...,False,137.0,126.0,133.0,48.0,50.5,50.0,70.0,70.00,75.0
832,26029,125054,200052,F,2193-09-18 12:19:00,2193-09-30 13:50:00,12.0632,57.4428,WHITE,EMERGENCY,...,True,110.0,113.0,114.0,59.0,57.5,58.5,77.0,76.50,77.5
6710,28494,199133,200059,F,2198-02-10 06:03:00,2198-02-23 13:45:00,13.3208,78.8991,WHITE,EMERGENCY,...,False,130.5,131.0,129.0,66.0,68.0,66.5,89.0,92.00,90.0
4313,99052,129142,200063,M,2141-03-09 23:19:00,2141-04-07 17:30:00,28.7576,37.0740,NaN,EMERGENCY,...,True,108.0,111.5,113.0,53.0,52.5,52.0,69.0,68.75,68.0


In [85]:
mimic_table2.head(2)

,icustay_id,charttime,heartrate,sysbp,diasbp,meanbp,resprate,tempc,spo2,glucose,intime,chartoffset
0,200003,2199-08-02 19:15:00,132.0,NaN,NaN,NaN,25.0,39.555554,97.0,NaN,2199-08-02 19:50:04,-35.066667
1,200003,2199-08-02 20:00:00,119.0,NaN,NaN,NaN,35.0,NaN,97.0,NaN,2199-08-02 19:50:04,9.933333


In [94]:
mimic_table2[(mimic_table2["icustay_id"]==200021) & (mimic_table2['diasbp'].notnull())]

,icustay_id,charttime,heartrate,sysbp,diasbp,meanbp,resprate,tempc,spo2,glucose,intime,chartoffset
868,200021,2114-12-26 22:29:00,NaN,94.0,63.0,76.0,NaN,NaN,NaN,NaN,2114-12-26 19:45:12,163.8
869,200021,2114-12-26 22:50:00,73.0,93.0,62.0,74.0,21.0,NaN,100.0,NaN,2114-12-26 19:45:12,184.8
870,200021,2114-12-26 23:00:00,65.0,94.0,61.0,75.0,43.0,35.222222,100.0,NaN,2114-12-26 19:45:12,194.8
871,200021,2114-12-27 00:00:00,71.0,96.0,61.0,75.0,14.0,35.777778,100.0,NaN,2114-12-26 19:45:12,254.8
872,200021,2114-12-27 01:00:00,76.0,96.0,63.0,76.0,15.0,NaN,99.0,NaN,2114-12-26 19:45:12,314.8
874,200021,2114-12-27 02:00:00,78.0,101.0,64.0,78.0,14.0,36.888889,99.0,NaN,2114-12-26 19:45:12,374.8
875,200021,2114-12-27 03:00:00,92.0,124.0,75.0,93.0,15.0,NaN,100.0,NaN,2114-12-26 19:45:12,434.8
877,200021,2114-12-27 04:00:00,98.0,124.0,72.0,91.0,17.0,NaN,100.0,NaN,2114-12-26 19:45:12,494.8
878,200021,2114-12-27 05:00:00,96.0,125.0,73.0,90.0,18.0,NaN,100.0,NaN,2114-12-26 19:45:12,554.8
879,200021,2114-12-27 06:00:00,105.0,126.0,77.0,94.0,21.0,NaN,100.0,NaN,2114-12-26 19:45:12,614.8


In [71]:
mimic_result[["icustay_id", "ibp_diastolic_24h"]].head(10)

,icustay_id,ibp_diastolic_24h
8839,200003,62.5
4325,200009,57.0
4241,200014,63.5
8867,200025,60.0
2705,200028,46.0
5534,200030,65.0
11249,200039,48.0
832,200052,59.0
6710,200059,66.0
4313,200063,53.0


In [36]:
print(ibp_sys_mean_24.head(1))
print(ibp_dias_mean_24.head(1))
print(ibp_mean_mean_24.head(1))

patientunitstayid
141168   NaN
dtype: float64
patientunitstayid
141168   NaN
dtype: float64
patientunitstayid
141168    78.3
dtype: float64


In [37]:
print(ibp_sys_mean_48.head(1))
print(ibp_dias_mean_48.head(1))
print(ibp_mean_mean_48.head(1))

patientunitstayid
141168    104.8
dtype: float64
patientunitstayid
141168    52.0
dtype: float64
patientunitstayid
141168    75.0
dtype: float64


In [38]:
print(ibp_sys_mean_72.head(1))
print(ibp_dias_mean_72.head(1))
print(ibp_mean_mean_72.head(1))

patientunitstayid
141168    104.8
dtype: float64
patientunitstayid
141168    52.0
dtype: float64
patientunitstayid
141168    75.0
dtype: float64


In [43]:
BP_data_table2['chartoffset'].min()

-229083

# vassopressor duration

In [21]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password='eicus')
cur = con.cursor()
cur.execute('SET search_path to ' + 'public')

query4 = \
"""
with vassopressor as
(select patientunitstayid, drugorderoffset, chartoffset, drugstopoffset,
case when (norepinephrine +  epinephrine + dopamine + phenylephrine + vasopressin) > 0 then 1 else 0 end as vasso_ind
from bp_project.pivoted_med
)


select * from vassopressor
where vasso_ind = 1
order by patientunitstayid, chartoffset




"""

vasso = pd.read_sql_query(query4,con)
vasso.head()


,patientunitstayid,drugorderoffset,chartoffset,drugstopoffset,vasso_ind
0,141168,2048.0,2046,2390.0,1
1,141168,2121.0,2121,2390.0,1
2,141194,1449.0,1452,4847.0,1
3,141233,8.0,2,2881.0,1
4,141233,7.0,17,1456.0,1


In [23]:
vasso.to_csv("vasso_duration_raw.csv")

In [37]:
df = pd.read_csv("vasso_duration_raw.csv", index_col = 0)

In [41]:
# create start_time and end_time for each 24, 48, 72 hours interval
df['start_time24'] = df['chartoffset']
df['start_time48'] = df['chartoffset']
df['start_time72'] = df['chartoffset']

df['end_time24'] = df['drugstopoffset']
df['end_time48'] = df['drugstopoffset']
df['end_time72'] = df['drugstopoffset']


df.loc[df.chartoffset > 1440, 'start_time24'] = 1440
df.loc[df.chartoffset > 2880, 'end_time48'] = 2880
df.loc[df.chartoffset > 4320, 'end_time72'] = 4320
df.loc[df.chartoffset < 0, 'start_time24'] = 0
df.loc[df.chartoffset < 0, 'end_time48'] = 0
df.loc[df.chartoffset < 0, 'end_time72'] = 0

df.loc[df.drugstopoffset > 1440, 'end_time24'] = 1440
df.loc[df.drugstopoffset > 2880, 'end_time48'] = 2880
df.loc[df.drugstopoffset > 4320, 'end_time72'] = 4320

df.loc[df.chartoffset < 0, 'end_time24'] = 0
df.loc[df.chartoffset < 0, 'end_time48'] = 0
df.loc[df.chartoffset < 0, 'end_time72'] = 0

In [39]:
df = pd.DataFrame(df)
df.head()

,patientunitstayid,drugorderoffset,chartoffset,drugstopoffset,vasso_ind,start_time24,start_time48,start_time72,end_time24,end_time48,end_time72
0,141168,2048.0,2046,2390.0,1,1440,2046,2046,1440.0,2390.0,2390.0
1,141168,2121.0,2121,2390.0,1,1440,2121,2121,1440.0,2390.0,2390.0
2,141194,1449.0,1452,4847.0,1,1440,1452,1452,1440.0,2880.0,4320.0
3,141233,8.0,2,2881.0,1,2,2,2,1440.0,2880.0,2881.0
4,141233,7.0,17,1456.0,1,17,17,17,1440.0,1456.0,1456.0


In [265]:
df_test = df_dropped.copy().iloc[:1000]

In [205]:
df_test.sort_values('chartoffset', inplace=True)

In [207]:
df_test.loc[df_test.chartoffset < 0, 'chartoffset'] = 0
df_test.loc[df_test.drugstopoffset < 0, 'drugstopoffset'] = 0

In [233]:
check = df[df['chartoffset']>df['drugstopoffset']]
(check['chartoffset']-check['drugstopoffset']).sort_values(ascending=False)

51703    3539.0
51702    3537.0
48652    1444.0
60911    1444.0
50529    1442.0
49957    1431.0
69035    1426.0
67642    1426.0
79252    1389.0
70344    1254.0
61412    1246.0
67846    1245.0
44006    1230.0
5391     1203.0
61413    1195.0
39940    1192.0
69058    1116.0
68847    1110.0
47889    1107.0
48663    1102.0
10103    1074.0
3108     1050.0
49981    1040.0
78979    1008.0
36945    1007.0
65149     998.0
5811      992.0
17075     991.0
47934     990.0
48662     982.0
          ...  
3214        1.0
3126        1.0
2659        1.0
2647        1.0
5400        1.0
5421        1.0
5943        1.0
5445        1.0
5834        1.0
5829        1.0
5805        1.0
5756        1.0
5752        1.0
5716        1.0
5715        1.0
5714        1.0
5676        1.0
5670        1.0
5669        1.0
5652        1.0
5640        1.0
5618        1.0
5611        1.0
5608        1.0
5604        1.0
5578        1.0
5481        1.0
5452        1.0
5451        1.0
7           1.0
Length: 2021, dtype: flo

In [241]:
df.shape

(84037, 12)

In [247]:
df_copy = df.copy()
df_copy.loc[df_copy.chartoffset < 0, 'chartoffset'] = 0
df_copy.loc[df_copy.drugstopoffset < 0, 'drugstopoffset'] = 0

In [260]:
df_copy = df_copy.dropna()

In [261]:
df_dropped = df_copy[df_copy['chartoffset']<=df_copy['drugstopoffset']]
df_dropped_check = df_copy[df_copy['chartoffset']>df_copy['drugstopoffset']]

In [263]:
df_dropped.shape

(80588, 12)

In [264]:
df_dropped_check.shape

(1244, 12)

In [262]:
df_copy.shape

(81832, 12)

In [279]:
df_dropped['start_end'] = df_dropped[['chartoffset', 'drugstopoffset']].apply(list, axis=1)

/home/ephlius/.virtualenvs/3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [268]:
def find_interval(data):    
    intervals = data['start_end'].values
    if len(intervals)==1:
        merged = intervals
    else:
        merged = merge_intervals(intervals)    
    merged = merge_intervals(intervals) 
    result = pd.DataFrame({'start': [i[0] for i in merged], 
                           'end': [i[1] for i in merged],
                           'group': list(range(len(merged)))})
    return result

In [275]:
def merge_intervals(intervals):
    sorted_intervals = sorted(intervals, key=lambda x: x[0])
#     print(sorted_intervals)
    interval_index = 0
    #print(sorted_intervals)
#     print(len(sorted_intervals))
    for  i in sorted_intervals:

        if i[0] > sorted_intervals[interval_index][1]:
            interval_index += 1
            sorted_intervals[interval_index] = i
        else:
            sorted_intervals[interval_index] = [sorted_intervals[interval_index][0], i[1]]
    #print(sorted_intervals)
    return sorted_intervals[:interval_index+1]

In [276]:
# df_test.start_end.values

In [281]:
df_interval = df_dropped.groupby("patientunitstayid").apply(find_interval)

In [284]:
df_interval = df_interval.reset_index()

In [290]:
df_interval.head(10)

,patientunitstayid,level_1,start,end,group
0,141168,0,2046.0,2390.0,0
1,141194,0,1452.0,4847.0,0
2,141233,0,2.0,1456.0,0
3,141233,1,2057.0,10199.0,1
4,141244,0,0.0,0.0,0
5,141288,0,0.0,3100.0,0
6,141313,0,96.0,215.0,0
7,141314,0,38.0,196.0,0
8,141314,1,218.0,228.0,1
9,141314,2,233.0,1050.0,2


In [301]:
def cum_time(data, threshold):
    select = data[data['start']<=threshold]
    temp_end = [min(i, threshold) for i in select['end']]
    temp_time = sum([end - select['start'].values[idx] for idx, end in enumerate(temp_end)])
    result = pd.DataFrame({"Time": temp_time}, index=[0])
    return result

In [303]:
df_24hrs = df_interval.groupby('patientunitstayid').apply(lambda x: cum_time(x, threshold=24*60))
df_48hrs = df_interval.groupby('patientunitstayid').apply(lambda x: cum_time(x, threshold=48*60))
df_72hrs = df_interval.groupby('patientunitstayid').apply(lambda x: cum_time(x, threshold=72*60))

In [312]:
final = pd.merge(pd.merge(df_24hrs, 
                 df_48hrs, 
                 how='inner', on="patientunitstayid"), df_72hrs, how='inner', on="patientunitstayid")
final.columns = ['24hrs', '48hrs', '72hrs']

In [314]:
final.head()

,24hrs,48hrs,72hrs
patientunitstayid,,,
141168,0.0,344.0,344.0
141194,0.0,1428.0,2868.0
141233,1438.0,2277.0,3717.0
141244,0.0,0.0,0.0
141288,1440.0,2880.0,3100.0


In [316]:
final['24hrs_hr'] = final['24hrs'].apply(lambda x: round(x/60, 2))
final['48hrs_hr'] = final['48hrs'].apply(lambda x: round(x/60, 2))
final['72hrs_hr'] = final['72hrs'].apply(lambda x: round(x/60, 2))

In [319]:
final[["24hrs_hr", "48hrs_hr", "72hrs_hr"]].to_csv("./eicu_vassopressor_duration.csv")

# SOFA score

In [386]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password='eicus')
cur = con.cursor()
cur.execute('SET search_path to ' + 'bp_project')

query0 = \
"""
select * from pivoted_sofa
"""

BP_sofa = pd.read_sql_query(query0,con)
BP_sofa.head()


,patientunitstayid,sofatotal
0,141168,8
1,141178,2
2,141179,4
3,141194,3
4,141196,2


In [387]:
BP_sofa.to_csv("./eicu_SOFA.csv")

# following codes not used 

## cohort

In [384]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password='eicus')
cur = con.cursor()
cur.execute('SET search_path to ' + 'public')

query0 = \
"""
with aa as (
select *,((outputtotal::varchar ~ '^-?[0-9]+\.?[0-9]*$'))as flag
from intakeoutput
)
select * from aa
where flag = False
limit 100;
"""

BP_data_table0 = pd.read_sql_query(query0,con)
BP_data_table0.head()


,intakeoutputid,patientunitstayid,intakeoutputoffset,intaketotal,outputtotal,dialysistotal,nettotal,intakeoutputentryoffset,cellpath,celllabel,cellvaluenumeric,cellvaluetext,flag


In [385]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password='eicus')
cur = con.cursor()
cur.execute('SET search_path to ' + 'bp_project')

query0 = \
"""
select * from pivoted_sofa
limit 5
"""

BP_data_table0 = pd.read_sql_query(query0,con)
BP_data_table0.head()


,patientunitstayid,sofatotal
0,141168,8
1,141178,2
2,141179,4
3,141194,3
4,141196,2


In [363]:
BP_data_table0['respchartvalue'].unique()

array(['Continued', 'Off', 'Start', 'Suspended'], dtype=object)

In [325]:
BP_data_table0['nursingchartcelltypevalname'].unique()

array(['GCS Total', 'Verbal', 'Eyes', 'Motor'], dtype=object)

In [85]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password='eicus')
cur = con.cursor()
cur.execute('SET search_path to ' + 'public')

query0 = \
"""
with cohort_raw as (
select pa.patientHealthSystemStayID, pa.patientunitstayid, pa.hospitalAdmitOffset,
Row_number() OVER (PARTITION BY patientHealthSystemStayID
                             ORDER BY pa.hospitalAdmitOffset DESC) AS POSITION
from eicu.patient pa
order by patientHealthSystemStayID, patientunitstayid
),

-- ############ first ICU admission ############ 
cohort_first as (
select * from cohort_raw
where cohort_raw.POSITION =1
),
-- ############ BP reading in the first 24h and has fulfilled criteria 1,2,3 ############
cohort as (
select cohort_first.patientunitstayid
from cohort_first inner join BP_project.BP_cohort
on cohort_first.patientunitstayid = BP_cohort.patientunitstayid
)

select * from cohort


"""

BP_data_table0 = pd.read_sql_query(query0,con)
BP_data_table0.head()


,patientunitstayid
0,141168
1,141194
2,141203
3,141229
4,141233


In [86]:
BP_data_table0.shape

(22518, 1)

In [132]:
# create a database connection
sqluser = 'ephlius'
dbname = 'mimic'
schema_name = 'mimiciii'

con = psycopg2.connect(dbname=dbname, user=sqluser, password='mimic')
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

query0 = \
"""
select distinct icd9_code,  short_title  from d_icd_diagnoses
where lower(short_title) like '%hypertension%';


"""

BP_data_table0 = pd.read_sql_query(query0,con)
BP_data_table0.head()


,icd9_code,short_title
0,36504,Ocular hypertension
1,4010,Malignant hypertension
2,4011,Benign hypertension
3,4019,Hypertension NOS
4,40591,Renovasc hypertension


In [114]:
BP_data_table0.head(20)

,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
0,12716,220050,Arterial Blood Pressure systolic,ABPs,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,None
